### Import libraries

In [1]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from collections import defaultdict

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Load data

In [2]:
launches_file = '../data/raw/launch/launch_weather_stats.csv'
weather_file  = '../data/transformed/weather/cape_canaveral_usa_hourly.csv'
launch_violations_file = '../data/raw/launch/clean_launch_stats.csv'
field_mill_file = '/Volumes/Extreme SSD/Git/ksc-weather-retriever/data/integrated/field_mill_50hz/fieldmill_hourly_all_years.csv'
wind_profile_file = '/Volumes/Extreme SSD/Git/ksc-weather-retriever/data/integrated/wind_profiler/wind_profiler_hourly_all_years.csv'

In [3]:
# Load launch data by: 
# - parsing dates
# - converting launch vehicle, payload, and remarks to strings
# - replacing spaces, NaNs, and NAs with None
launches = pd.read_csv(launches_file,
                       parse_dates=['Date'],
                       dtype={'Launch_Vehicle': str, 'Payload': str, 'Remarks': str},
                       na_values=[' ', 'NaN', 'NA'])
launches.columns = launches.columns.str.upper()
launches['DATE'] = pd.to_datetime(launches['DATE'], errors='coerce')

In [4]:
# Load weather data by:
# - parsing dates
# - converting weather code to string
print("Loading weather data...")
weather = pd.read_csv(weather_file, parse_dates=['time'], dtype={'weather_code (wmo code)': str}).rename(columns=lambda x: x.upper())
print("Weather data loaded successfully")

print("Loading field mill data...")
field_mill = pd.read_csv(field_mill_file, parse_dates=['datetime']).rename(columns=lambda x: x.upper())
field_mill['DATETIME'] = pd.to_datetime(field_mill['DATETIME'])
print("Field mill data loaded successfully")

print("Loading wind profile data...")
wind_profiles = pd.read_csv(wind_profile_file, parse_dates=['datetime']).rename(columns=lambda x: x.upper())
wind_profiles['DATETIME'] = pd.to_datetime(wind_profiles['DATETIME'])
print("Wind profile data loaded successfully")

Loading weather data...
Weather data loaded successfully
Loading field mill data...
Field mill data loaded successfully
Loading wind profile data...
Wind profile data loaded successfully


In [34]:
wind_profiles.head()

,DATETIME,ALTITUDE,DIRECTION_MEAN,DIRECTION_MEDIAN,DIRECTION_MAX,DIRECTION_MIN,DIRECTION_COUNT,SPEED_MEAN,SPEED_MEDIAN,SPEED_MAX,SPEED_MIN,SPEED_COUNT,SHEAR_MEAN,SHEAR_MEDIAN,SHEAR_MAX,SHEAR_MIN,SHEAR_COUNT,VERTICAL_VELOCITY_MEAN,VERTICAL_VELOCITY_MEDIAN,VERTICAL_VELOCITY_MAX,VERTICAL_VELOCITY_MIN,VERTICAL_VELOCITY_COUNT
0,2018-01-01,1795,281.333333,281.0,288,279,12,29.154785,29.080220,32.435630,26.395892,12,0.000000,0.000,0.000,0.000,12,0.073333,0.090,0.17,-0.10,12
1,2018-01-01,1944,281.166667,281.0,284,280,12,31.317160,31.317160,32.883018,29.527608,12,0.006750,0.007,0.011,0.002,12,0.059167,0.075,0.16,-0.14,12
2,2018-01-01,2093,283.416667,283.5,287,282,12,29.639455,29.751302,31.093466,27.066974,12,0.006333,0.006,0.009,0.004,12,0.096667,0.110,0.17,-0.09,12
3,2018-01-01,2243,281.833333,282.0,286,280,12,30.944337,31.093466,32.435630,29.080220,12,0.004750,0.005,0.007,0.002,12,0.094167,0.100,0.18,-0.12,12
4,2018-01-01,2392,280.583333,281.0,282,279,12,31.168031,30.981619,32.883018,29.527608,12,0.003250,0.003,0.005,0.002,12,0.067500,0.080,0.17,-0.13,12


In [5]:
# Load launch violations data by:
# - dropping image columns
launch_violations = pd.read_csv(launch_violations_file).rename(columns=lambda x: x.upper())
img_cols = ['GOES_IMG_PATH', 'SHEAR_IMG_PATH', 'WARNIGN_IMG_PATH', 'SBCAPE_CIN_IMG_PATH']
launch_violations = launch_violations.drop(columns=img_cols)

### Clean data

Convert binary columns to 1/0 values and fill missing values in PAYLOAD/REMARKS

In [6]:
# Convert columns to binary (1 for 1.0, 0 for NaN)
binary_columns = ['COUNTDOWN', 'LAUNCHED', 'NON_WX_SCRUB', 'WX_SCRUB', 
                 'NON_WX_DELAY', 'WX DELAY', 'LCC_SCRUB_DELAY', 'USER_WX_SCRUB_DELAY']

launches[binary_columns] = launches[binary_columns].fillna(0).astype(int)

# Fill NaN values in PAYLOAD and REMARKS columns
launches['PAYLOAD'] = launches['PAYLOAD'].fillna('N/A')
launches['REMARKS'] = launches['REMARKS'].fillna('NONE')

Drop columns from launch_violations where the column names contain “DURATION”, “COUNT”, or “VIOLATED”

In [7]:
launch_violations = launch_violations.drop(columns=[col for col in launch_violations.columns if 'DURATION' in col or 'COUNT' in col or 'VIOLATED' in col])

Classify launch notes into categories like “clouds”, “wind”, “lightning”, etc., based on keyword matching. If no match is found, it checks for “delay” or “scrub” and returns “other”.

In [8]:
def categorize_launch_notes(notes):
    """
    Categorizes launch notes into simplified explanations.

    Args:
        notes: A string containing the launch note.

    Returns:
        A string representing the category of the note.
    """
    # Special case for "NONE"
    if notes == "NONE":
        return "none"

    # Define categories with their keywords, ordered by priority
    categories = {
        "clouds": [
            "cumulus", "Cu Cloud", "Cu cloud", "cloud",
            "anvil", "Anvil", "Detached Anvil", "attached anvil", 
            "Thickcloud", "Thick Cloud", "CLOUDS", "clouds", "CLDS",
            "CLD"
        ],
        "wind": [
            "wind", "Wind", "Wx", "WX",
            "upper level wind", "Upper Level Wind",
            "ground wind", "winds", "LOADS"
        ],
        "lightning": [
            "lightning", "field mill", "Field Mill", 
            "Surface Electric Fields", "LLCC",
            "electric field", "LTG", "THUNDERSTORM"
        ],
        "range": [
            "range", "Range", "airspace", "restricted area", 
            "radar", "FTS", "helicopter", "cruise ship",
            "tracking", "recovery"
        ],
        "vehicle": [
            "Falcon", "vehicle", "Vehicle", "engine", "leak",
            "valve", "hydrogen", "helium", "propellant", "LV",
            "stage", "tank", "cryo", "H2", "instrumentation",
            "hardware"
        ],
        "ground_systems": [
            "Ground system", "ground", "ULA equip", 
            "user anomaly", "comms", "camera", 
            "mission assurance", "mission readiness",
            "customer ground systems", "ground systems",
            "data review"
        ],
        "weather_other": [
            "weather", "Weather", "rain", "shower", "icing"
        ],
        "payload": [
            "payload", "Payload", "user", "customer",
            "spacecraft", "S/C"
        ],
        "drone": ["Droneship", "drone", "recovery ship"],
        "abort": ["abort", "Abort"],
        "other": ["Technical issue", "Tracking issues", "Shift of T-0"]
    }

    # Convert note to lowercase for case-insensitive matching
    note_lower = notes.lower()

    # First check for exact phrases that might need special handling
    if "upper level wind" in note_lower:
        return "wind"
    if "second stage" in note_lower or "2nd stage" in note_lower:
        return "vehicle"
    if "t-0" in note_lower and "delay" in note_lower:
        # Look for specific reason after "due to"
        if "due to" in note_lower:
            reason = note_lower.split("due to")[1].strip()
            # Recursively categorize the reason
            return categorize_launch_notes(reason)
        return "other"

    # Check each category's keywords
    for category, keywords in categories.items():
        for keyword in keywords:
            if re.search(rf"(?<!\w){re.escape(keyword.lower())}(?!\w)", note_lower):
                return category

    # If no category found, check for delay/scrub
    if re.search(r"delay|scrub", note_lower):
        return "other"

    return "unknown"

launches['REMARKS_CATEGORY'] = launches['REMARKS'].apply(categorize_launch_notes)

In [9]:
launches['REMARKS_CATEGORY'].value_counts()

REMARKS_CATEGORY
none              403
unknown           146
wind               81
vehicle            53
clouds             47
range              31
lightning          22
other              15
payload            11
ground_systems      9
abort               7
weather_other       4
drone               1
Name: count, dtype: int64

Converts wind speeds from meters per second (m/s) to miles per hour (mph) for SPEED_MEAN, SPEED_MEDIAN, SPEED_MAX, and SPEED_MIN.

In [10]:
# Convert wind speeds from m/s to mph
wind_profiles['SPEED_MEAN'] = wind_profiles['SPEED_MEAN'] * 2.23694
wind_profiles['SPEED_MEDIAN'] = wind_profiles['SPEED_MEDIAN'] * 2.23694
wind_profiles['SPEED_MAX'] = wind_profiles['SPEED_MAX'] * 2.23694
wind_profiles['SPEED_MIN'] = wind_profiles['SPEED_MIN'] * 2.23694

# Convert wind shear (/sec) to 

Bin wind profile data into atmospheric layers (e.g., boundary layer, troposphere, stratosphere) based on altitude, aggregates measurements (e.g., direction, speed, shear) by hour, and renames columns for clarity.

In [12]:
def bin_wind_profiles_by_layer(df):
    """
    Bin wind profile data into atmospheric layers and aggregate measurements hourly.
    """
    bins = [0, 1000, 3000, 5000, 7000, 9000, 10000, 12000, 15000, 16000, 18000, float('inf')]
    labels = ['surface_layer_1000hpa', 'boundary_layer_900hpa', 'low_troposphere_700hpa', 'mid_low_troposphere_500hpa', 'mid_troposphere_400hpa',
              'high_mid_troposphere_300hpa', 'upper_troposphere_200hpa', 'lower_stratosphere_150hpa', 'mid_stratosphere_100hpa',
              'upper_stratosphere_50hpa', 'top_stratosphere_10hpa']

    df = df.copy()
    df['ATMOSPHERIC_LAYER'] = pd.cut(df['ALTITUDE'], bins=bins, labels=labels)

    # Restructure measurements to avoid redundant names
    measurement_cols = {
        # 'DIRECTION_MEAN': ['mean', 'max'],
        'SPEED_MEAN': ['mean', 'max'],
        'SHEAR_MEAN': ['mean', 'max'],
        # 'DIRECTION_MAX': ['mean', 'max'],
        'SPEED_MAX': ['mean', 'max'],
        'SHEAR_MAX': ['mean', 'max'],
    }

    grouped = df.groupby([
        pd.Grouper(key='DATETIME', freq='H'),
        'ATMOSPHERIC_LAYER'
    ]).agg(measurement_cols)

    # Rename columns to be more concise
    new_names = {
        # 'DIRECTION_MEAN_mean': 'DIRECTION_AVG',
        # 'DIRECTION_MEAN_max': 'DIRECTION_AVG_PEAK',
        'SPEED_MEAN_mean': 'SPEED_AVG',
        'SPEED_MEAN_max': 'SPEED_AVG_PEAK',
        'SHEAR_MEAN_mean': 'SHEAR_AVG',
        'SHEAR_MEAN_max': 'SHEAR_AVG_PEAK',
        # 'DIRECTION_MAX_mean': 'DIRECTION_PEAK_AVG',
        # 'DIRECTION_MAX_max': 'DIRECTION_MAX_PEAK',
        'SPEED_MAX_mean': 'SPEED_MAX_AVG',
        'SPEED_MAX_max': 'SPEED_MAX_PEAK',
        'SHEAR_MAX_mean': 'SHEAR_MAX_AVG',
        'SHEAR_MAX_max': 'SHEAR_MAX_PEAK',
    }

    # Flatten and rename columns
    grouped.columns = [f'{col[0]}_{col[1]}' for col in grouped.columns]
    grouped = grouped.reset_index()
    grouped = grouped.rename(columns=new_names)

    return grouped

# Bin wind profiles by atmospheric layer
binned_wind_profiles = bin_wind_profiles_by_layer(wind_profiles)

/var/folders/cl/cmd4ttnn79d91jcrl066qh4r0000gn/T/ipykernel_34097/3267685395.py:24: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pd.Grouper(key='DATETIME', freq='H'),
/var/folders/cl/cmd4ttnn79d91jcrl066qh4r0000gn/T/ipykernel_34097/3267685395.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby([


### Merge data

Convert DATE columns to datetime for consistent merging, filters data from 2018 onwards, merges launches and launch_violations on DATE, reorders columns, renames and drops specific columns.

In [13]:
# Convert launch_violations DATE to datetime for consistent merging
launch_violations['DATE'] = pd.to_datetime(launch_violations['DATE'])

# Convert launches DATE to datetime.date to match launch_violations
launches['DATE'] = launches['DATE'].dt.date
launch_violations['DATE'] = launch_violations['DATE'].dt.date

# Filter data from 2018 onwards
launches = launches[launches['DATE'] >= pd.to_datetime('2018-01-01').date()]
launch_violations = launch_violations[launch_violations['DATE'] >= pd.to_datetime('2018-01-01').date()]

# Merge launches and launch_violations datasets on date
# Reorder columns to keep START_LCC_EVAL and END_LCC_EVAL next to DATE
merged_df = pd.merge(
    launches,
    launch_violations,
    on='DATE',
    how='left'
)

# Reorder columns to put evaluation times next to DATE
date_cols = ['DATE', 'START_LCC_EVAL', 'END_LCC_EVAL']
other_cols = [col for col in merged_df.columns if col not in date_cols]
merged_df = merged_df[date_cols + other_cols]

# Rename 'LAUNCH_VEHICLE_x' and 'PAYLOAD_x' to 'LAUNCH_VEHICLE' and 'PAYLOAD'
merged_df = merged_df.rename(columns={'LAUNCH_VEHICLE_x': 'LAUNCH_VEHICLE', 'PAYLOAD_x': 'PAYLOAD'})

# Drop the columns 'LAUNCH_VEHICLE_y' and 'PAYLOAD_y' from the merged dataframe
merged_df = merged_df.drop(columns=['LAUNCH_VEHICLE_y', 'PAYLOAD_y'])

Filter merged_df to keep rows with indexes specified in indexes_to_keep and all non-indexes_to_keep rows that are unique by the ‘DATE’ column, removing unwanted duplicates.

In [14]:
# Keep only specific indexes from duplicates by:
# - Keeping the indexes that are in the indexes_to_keep list
# - Keeping all duplicates that are not in the indexes_to_keep list
indexes_to_keep = [181, 184, 209, 212, 229, 232]
merged_df = merged_df.loc[merged_df.index.isin(indexes_to_keep) | ~merged_df.duplicated(subset=['DATE'], keep=False)]

In [15]:
merged_df.to_csv('../data/transformed/launch/cape_canaveral_launches.csv', index=False)

Process weather data by converting relevant columns to datetime, filtering weather records for each launch’s 8-hour window leading up to the launch, and padding any missing data with zeros if necessary. For each launch, the function compiles weather features into arrays. These arrays are then organized into a dataframe, which can be used for further analysis.

In [16]:
def create_weather_arrays(merged_df, weather_df):
    '''
    Create a dataframe of weather arrays for each launch by:
    - Converting END_LCC_EVAL and TIME to datetime
    - Converting weather TIME from UTC to Eastern time
    - Looping through each launch and appending the weather data for the 8 hours before the launch to the lists
    - Creating a dataframe from the lists
    '''
    # Convert END_LCC_EVAL to datetime if it's not already
    merged_df['END_LCC_EVAL'] = pd.to_datetime(merged_df['END_LCC_EVAL'])
    merged_df['START_LCC_EVAL'] = pd.to_datetime(merged_df['START_LCC_EVAL'])
    merged_df['DATE'] = pd.to_datetime(merged_df['DATE'])
    weather_df['TIME'] = pd.to_datetime(weather_df['TIME'])

    # Initialize lists to store our results
    all_dates = []
    all_start_times = []
    all_end_times = []
    all_temps = []
    all_cloud_cover = []
    all_wind_speed_10m = []
    all_wind_speed_100m = []
    all_wind_dir_10m = []
    all_wind_dir_100m = []
    all_wind_gusts = []
    all_pressure = []
    all_rain = []
    all_humidity = []
    all_weather_code = []

    # For each launch
    for _, row in merged_df.iterrows():
        end_time = row['END_LCC_EVAL']
        if pd.isna(end_time):
            continue

        # Get the 8 hours before end time
        start_time = end_time - pd.Timedelta(hours=8)

        # Filter weather data for this time window
        mask = (weather_df['TIME'] >= start_time) & (weather_df['TIME'] <= end_time)
        relevant_weather = weather_df[mask].sort_values('TIME', ascending=False)

        # Pad with zeros if less than 8 hours
        pad_length = 8 - len(relevant_weather)

        temps = relevant_weather['TEMPERATURE_2M (°F)'].tolist() + [0] * pad_length
        cloud = relevant_weather['CLOUD_COVER (%)'].tolist() + [0] * pad_length
        wind_10m = relevant_weather['WIND_SPEED_10M (MP/H)'].tolist() + [0] * pad_length
        wind_100m = relevant_weather['WIND_SPEED_100M (MP/H)'].tolist() + [0] * pad_length
        wind_dir_10m = relevant_weather['WIND_DIRECTION_10M (°)'].tolist() + [0] * pad_length
        wind_dir_100m = relevant_weather['WIND_DIRECTION_100M (°)'].tolist() + [0] * pad_length
        gusts = relevant_weather['WIND_GUSTS_10M (MP/H)'].tolist() + [0] * pad_length
        pressure = relevant_weather['PRESSURE_MSL (HPA)'].tolist() + [0] * pad_length
        rain = relevant_weather['RAIN (INCH)'].tolist() + [0] * pad_length
        humidity = relevant_weather['RELATIVE_HUMIDITY_2M (%)'].tolist() + [0] * pad_length
        weather_code = relevant_weather['WEATHER_CODE (WMO CODE)'].tolist() + [0] * pad_length

        # Append to our lists
        all_dates.append(row['DATE'])
        all_start_times.append(row['START_LCC_EVAL'])
        all_end_times.append(row['END_LCC_EVAL'])
        all_temps.append(temps[:8])  # Ensure we only take first 8 values
        all_cloud_cover.append(cloud[:8])
        all_wind_speed_10m.append(wind_10m[:8])
        all_wind_speed_100m.append(wind_100m[:8])
        all_wind_dir_10m.append(wind_dir_10m[:8])
        all_wind_dir_100m.append(wind_dir_100m[:8])
        all_wind_gusts.append(gusts[:8])
        all_pressure.append(pressure[:8])
        all_rain.append(rain[:8])
        all_humidity.append(humidity[:8])
        all_weather_code.append(weather_code[:8])

    # Create the final dataframe
    weather_arrays_df = pd.DataFrame({
        'DATE': all_dates,
        'START_LCC_EVAL': all_start_times,
        'END_LCC_EVAL': all_end_times,
        'TEMPERATURE': all_temps,
        'CLOUD_COVER': all_cloud_cover,
        'WIND_SPEED_10M': all_wind_speed_10m,
        'WIND_SPEED_100M': all_wind_speed_100m,
        'WIND_DIRECTION_10M': all_wind_dir_10m,
        'WIND_DIRECTION_100M': all_wind_dir_100m,
        'WIND_GUSTS': all_wind_gusts,
        'PRESSURE': all_pressure,
        'RAIN': all_rain,
        'HUMIDITY': all_humidity,
        'WEATHER_CODE': all_weather_code
    })

    return weather_arrays_df

# Create the weather arrays dataframe
weather_arrays = create_weather_arrays(merged_df, weather)

In [17]:
def create_field_mill_arrays(merged_df, field_mill_df):
    '''
    Create a dataframe of field mill arrays for each launch by:
    - Converting END_LCC_EVAL and DATETIME to datetime
    - Aggregating field mill data across sensors for each hour
    - Looping through each launch and appending the field mill data for the 8 hours before the launch
    - Creating a dataframe from the lists
    '''
    # Convert END_LCC_EVAL to datetime if it's not already
    merged_df['END_LCC_EVAL'] = pd.to_datetime(merged_df['END_LCC_EVAL'])
    field_mill_df['DATETIME'] = pd.to_datetime(field_mill_df['DATETIME'])

    # Initialize lists to store our results
    all_dates = []
    all_start_times = []
    all_end_times = []
    all_mean_values = []
    all_median_values = []
    all_max_values = []
    all_min_values = []

    # For each launch
    for _, row in merged_df.iterrows():
        end_time = row['END_LCC_EVAL']
        if pd.isna(end_time):
            continue

        # Get the 8 hours before end time
        start_time = end_time - pd.Timedelta(hours=8)

        # Filter field mill data for this time window
        mask = (field_mill_df['DATETIME'] >= start_time) & (field_mill_df['DATETIME'] <= end_time)
        relevant_data = field_mill_df[mask].copy()

        # Group by datetime and aggregate across sensors
        hourly_stats = relevant_data.groupby('DATETIME').agg({
            'MEAN': 'mean',
            'MEDIAN': 'mean',
            'MAX': 'max',
            'MIN': 'min'
        }).sort_values('DATETIME', ascending=False)

        # Pad with zeros if less than 8 hours
        pad_length = 8 - len(hourly_stats)

        means = hourly_stats['MEAN'].tolist() + [0] * pad_length
        medians = hourly_stats['MEDIAN'].tolist() + [0] * pad_length
        maxes = hourly_stats['MAX'].tolist() + [0] * pad_length
        mins = hourly_stats['MIN'].tolist() + [0] * pad_length

        # Append to our lists
        all_dates.append(row['DATE'])
        all_start_times.append(row['START_LCC_EVAL'])
        all_end_times.append(row['END_LCC_EVAL'])
        all_mean_values.append(means[:8])  # Ensure we only take first 8 values
        all_median_values.append(medians[:8])
        all_max_values.append(maxes[:8])
        all_min_values.append(mins[:8])

    # Create the final dataframe
    field_mill_arrays_df = pd.DataFrame({
        'DATE': all_dates,
        'START_LCC_EVAL': all_start_times,
        'END_LCC_EVAL': all_end_times,
        'FIELD_MILL_MEAN': all_mean_values,
        'FIELD_MILL_MEDIAN': all_median_values,
        'FIELD_MILL_MAX': all_max_values,
        'FIELD_MILL_MIN': all_min_values
    })

    return field_mill_arrays_df

# Create the field mill arrays dataframe
field_mill_arrays = create_field_mill_arrays(merged_df, field_mill)

In [19]:
def create_wind_profiler_arrays(merged_df, binned_wind_profiles_df):
    '''
    Create a dataframe of wind profiler arrays for each launch by:
    - Converting END_LCC_EVAL and DATETIME to datetime
    - Creating separate arrays for each atmospheric layer
    - Looping through each launch and appending the wind profiler data for the 8 hours before launch
    - Creating a dataframe from the lists
    '''
    # Convert END_LCC_EVAL to datetime if it's not already
    merged_df['END_LCC_EVAL'] = pd.to_datetime(merged_df['END_LCC_EVAL'])
    binned_wind_profiles_df['DATETIME'] = pd.to_datetime(binned_wind_profiles_df['DATETIME'])

    # Initialize lists for dates and each atmospheric layer's measurements
    all_dates = []
    all_start_times = []
    all_end_times = []
    layers = ['surface_layer_1000hpa', 'boundary_layer_900hpa', 'low_troposphere_700hpa', 'mid_low_troposphere_500hpa', 'mid_troposphere_400hpa',
            'high_mid_troposphere_300hpa', 'upper_troposphere_200hpa', 'lower_stratosphere_150hpa', 'mid_stratosphere_100hpa',
            'upper_stratosphere_50hpa', 'top_stratosphere_10hpa'
            ]

    # Dictionary to store arrays for each measurement type and layer
    measurements = {
        'SPEED_AVG': {layer: [] for layer in layers},
        'SPEED_MAX_PEAK': {layer: [] for layer in layers},
        'SHEAR_AVG': {layer: [] for layer in layers},
        'SHEAR_MAX_PEAK': {layer: [] for layer in layers}
    }

    # For each launch
    for _, row in merged_df.iterrows():
        end_time = row['END_LCC_EVAL']
        if pd.isna(end_time):
            continue

        # Get the 8 hours before end time
        start_time = end_time - pd.Timedelta(hours=8)

        # Filter wind profiler data for this time window
        mask = (binned_wind_profiles_df['DATETIME'] >= start_time) & (binned_wind_profiles_df['DATETIME'] <= end_time)
        relevant_data = binned_wind_profiles_df[mask].copy()

        # Add date and times to our lists
        all_dates.append(row['DATE'])
        all_start_times.append(row['START_LCC_EVAL'])
        all_end_times.append(row['END_LCC_EVAL'])

        # Process each atmospheric layer
        for layer in layers:
            layer_data = relevant_data[relevant_data['ATMOSPHERIC_LAYER'] == layer].sort_values('DATETIME', ascending=False)

            # Pad with zeros if less than 8 hours
            pad_length = 8 - len(layer_data)

            # Process each measurement type for this layer
            for measure in measurements.keys():
                values = layer_data[measure].tolist() + [0] * pad_length
                measurements[measure][layer].append(values[:8])  # Ensure we only take first 8 values

    # Create the final dataframe
    wind_profiler_arrays_df = pd.DataFrame({
        'DATE': all_dates,
        'START_LCC_EVAL': all_start_times,
        'END_LCC_EVAL': all_end_times
    })

    # Add columns for each measurement and layer combination
    for measure in measurements.keys():
        for layer in layers:
            col_name = f'WIND_{measure}_{layer.upper()}'
            wind_profiler_arrays_df[col_name] = measurements[measure][layer]

    return wind_profiler_arrays_df

# Create the wind profiler arrays dataframe
wind_profiler_arrays = create_wind_profiler_arrays(merged_df, binned_wind_profiles)

In [25]:
print(wind_profiler_arrays.head(10).to_markdown())

|    | DATE                | START_LCC_EVAL      | END_LCC_EVAL        | WIND_SPEED_AVG_SURFACE_LAYER_1000HPA     | WIND_SPEED_AVG_BOUNDARY_LAYER_900HPA                                                                                                                           | WIND_SPEED_AVG_LOW_TROPOSPHERE_700HPA                                                                                                                           | WIND_SPEED_AVG_MID_LOW_TROPOSPHERE_500HPA                                                                                                                       | WIND_SPEED_AVG_MID_TROPOSPHERE_400HPA                                                                                                                       | WIND_SPEED_AVG_HIGH_MID_TROPOSPHERE_300HPA                                                                                                                     | WIND_SPEED_AVG_UPPER_TROPOSPHERE_200HPA                                         

Merge launch data with weather, field mill, and wind profiler data based on the DATE column using inner joins, creating a unified dataframe, launch_corpus.

In [20]:
# First merge all the weather/sensor data together
print("Merging weather and field mill arrays...")
weather_combined = pd.merge(weather_arrays, field_mill_arrays, 
                          on=['DATE', 'START_LCC_EVAL', 'END_LCC_EVAL'], 
                          how='inner')
print(f"Successfully merged with {len(weather_combined)} matching records")

print("\nMerging with wind profiler arrays...")
weather_combined = pd.merge(weather_combined, wind_profiler_arrays,
                          on=['DATE', 'START_LCC_EVAL', 'END_LCC_EVAL'], 
                          how='inner')
print(f"Successfully merged with {len(weather_combined)} matching records")

# Then merge with launch data
print("\nMerging with launch data...")
launch_corpus = pd.merge(merged_df, weather_combined, 
                        on=['DATE', 'START_LCC_EVAL', 'END_LCC_EVAL'], 
                        how='inner')
print(f"Successfully merged with {len(launch_corpus)} matching records")

Merging weather and field mill arrays...
Successfully merged with 232 matching records

Merging with wind profiler arrays...
Successfully merged with 232 matching records

Merging with launch data...
Successfully merged with 232 matching records


### Exploratory data analysis

In [102]:
# Filter for successful and weather scrubbed launches
successful_launches = launch_corpus[launch_corpus['LAUNCHED'] == 1].copy()
wx_scrub_launches = launch_corpus[launch_corpus['WX_SCRUB'] == 1].copy()

# Calculate rain arrays for T-2 to T-0 for both groups
success_rain_arrays = np.vstack(successful_launches['RAIN'].apply(lambda x: x[-3:]))
scrub_rain_arrays = np.vstack(wx_scrub_launches['RAIN'].apply(lambda x: x[-3:]))

# Calculate rain statistics for T-2 to T-0 for both groups
success_percentiles = np.percentile(success_rain_arrays, [0, 25, 50, 75, 100], axis=0)
scrub_percentiles = np.percentile(scrub_rain_arrays, [0, 25, 50, 75, 100], axis=0)

print("Rain statistics by hour before launch:")
print(f"\nSuccessful launches (n={len(successful_launches)}):")
for i, hour in enumerate(['T-2', 'T-1', 'T-0']):
    print(f"\n{hour}:")
    print(f"Min (0th): {success_percentiles[0,i]:.3f} inches")
    print(f"25th: {success_percentiles[1,i]:.3f} inches")
    print(f"Median (50th): {success_percentiles[2,i]:.3f} inches")
    print(f"75th: {success_percentiles[3,i]:.3f} inches")
    print(f"Max (100th): {success_percentiles[4,i]:.3f} inches")

print(f"\nWeather scrubbed launches (n={len(wx_scrub_launches)}):")
for i, hour in enumerate(['T-2', 'T-1', 'T-0']):
    print(f"\n{hour}:")
    print(f"Min (0th): {scrub_percentiles[0,i]:.3f} inches")
    print(f"25th: {scrub_percentiles[1,i]:.3f} inches")
    print(f"Median (50th): {scrub_percentiles[2,i]:.3f} inches")
    print(f"75th: {scrub_percentiles[3,i]:.3f} inches")
    print(f"Max (100th): {scrub_percentiles[4,i]:.3f} inches")

Rain statistics by hour before launch:

Successful launches (n=174):

T-2:
Min (0th): 0.000 inches
25th: 0.000 inches
Median (50th): 0.000 inches
75th: 0.000 inches
Max (100th): 0.303 inches

T-1:
Min (0th): 0.000 inches
25th: 0.000 inches
Median (50th): 0.000 inches
75th: 0.000 inches
Max (100th): 0.272 inches

T-0:
Min (0th): 0.000 inches
25th: 0.000 inches
Median (50th): 0.000 inches
75th: 0.000 inches
Max (100th): 0.165 inches

Weather scrubbed launches (n=23):

T-2:
Min (0th): 0.000 inches
25th: 0.000 inches
Median (50th): 0.000 inches
75th: 0.024 inches
Max (100th): 0.390 inches

T-1:
Min (0th): 0.000 inches
25th: 0.000 inches
Median (50th): 0.004 inches
75th: 0.012 inches
Max (100th): 0.165 inches

T-0:
Min (0th): 0.000 inches
25th: 0.000 inches
Median (50th): 0.008 inches
75th: 0.014 inches
Max (100th): 0.043 inches


In [96]:
wind_shear_layers = [
    # 'WIND_SHEAR_MAX_PEAK_SURFACE_LAYER_1000HPA',
    'WIND_SHEAR_MAX_PEAK_BOUNDARY_LAYER_900HPA',
    'WIND_SHEAR_MAX_PEAK_LOW_TROPOSPHERE_700HPA',
    'WIND_SHEAR_MAX_PEAK_MID_LOW_TROPOSPHERE_500HPA',
    'WIND_SHEAR_MAX_PEAK_MID_TROPOSPHERE_400HPA',
    'WIND_SHEAR_MAX_PEAK_HIGH_MID_TROPOSPHERE_300HPA',
    'WIND_SHEAR_MAX_PEAK_UPPER_TROPOSPHERE_200HPA',
    'WIND_SHEAR_MAX_PEAK_LOWER_STRATOSPHERE_150HPA',
    'WIND_SHEAR_MAX_PEAK_MID_STRATOSPHERE_100HPA',
    'WIND_SHEAR_MAX_PEAK_UPPER_STRATOSPHERE_50HPA',
    'WIND_SHEAR_MAX_PEAK_TOP_STRATOSPHERE_10HPA'
]

# Filter for successful and weather scrubbed launches
successful_launches = launch_corpus[launch_corpus['LAUNCHED'] == 1].copy()
wx_scrub_launches = launch_corpus[launch_corpus['WX_SCRUB'] == 1].copy()

print("Maximum observed wind shear by layer and hour before launch:")
for layer in wind_shear_layers:
    layer_name = layer.replace('WIND_SHEAR_MAX_PEAK_', '').replace('_', ' ').title()
    
    # Successful launches
    success_t2 = successful_launches[layer].apply(lambda x: x[-3]).max()
    success_t1 = successful_launches[layer].apply(lambda x: x[-2]).max()
    success_t0 = successful_launches[layer].apply(lambda x: x[-1]).max()
    
    # Weather scrubbed launches  
    scrub_t2 = wx_scrub_launches[layer].apply(lambda x: x[-3]).max()
    scrub_t1 = wx_scrub_launches[layer].apply(lambda x: x[-2]).max()
    scrub_t0 = wx_scrub_launches[layer].apply(lambda x: x[-1]).max()
    
    print(f"\n{layer_name}:")
    print("Successful launches:")
    print(f"T-2: {success_t2:.3f}")
    print(f"T-1: {success_t1:.3f}")
    print(f"T-0: {success_t0:.3f}")
    print("Weather scrubbed launches:")
    print(f"T-2: {scrub_t2:.3f}")
    print(f"T-1: {scrub_t1:.3f}")
    print(f"T-0: {scrub_t0:.3f}")

Maximum observed wind shear by layer and hour before launch:

Boundary Layer 900Hpa:
Successful launches:
T-2: 0.071
T-1: 0.074
T-0: 0.073
Weather scrubbed launches:
T-2: 0.070
T-1: 0.064
T-0: 0.049

Low Troposphere 700Hpa:
Successful launches:
T-2: 0.099
T-1: 0.089
T-0: 0.100
Weather scrubbed launches:
T-2: 0.091
T-1: 0.035
T-0: 0.058

Mid Low Troposphere 500Hpa:
Successful launches:
T-2: 0.106
T-1: 0.060
T-0: 0.056
Weather scrubbed launches:
T-2: 0.080
T-1: 0.045
T-0: 0.041

Mid Troposphere 400Hpa:
Successful launches:
T-2: 0.113
T-1: 0.055
T-0: 0.066
Weather scrubbed launches:
T-2: 0.080
T-1: 0.040
T-0: 0.037

High Mid Troposphere 300Hpa:
Successful launches:
T-2: 0.149
T-1: 0.095
T-0: 0.075
Weather scrubbed launches:
T-2: 0.056
T-1: 0.044
T-0: 0.035

Upper Troposphere 200Hpa:
Successful launches:
T-2: 0.118
T-1: 0.102
T-0: 0.144
Weather scrubbed launches:
T-2: 0.109
T-1: 0.092
T-0: 0.067

Lower Stratosphere 150Hpa:
Successful launches:
T-2: 0.123
T-1: 0.105
T-0: 0.519
Weather scrub

In [95]:
wind_speed_layers = [
    'WIND_SPEED_MAX_PEAK_BOUNDARY_LAYER_900HPA',
    'WIND_SPEED_MAX_PEAK_LOW_TROPOSPHERE_700HPA',
    'WIND_SPEED_MAX_PEAK_MID_LOW_TROPOSPHERE_500HPA',
    'WIND_SPEED_MAX_PEAK_MID_TROPOSPHERE_400HPA',
    'WIND_SPEED_MAX_PEAK_HIGH_MID_TROPOSPHERE_300HPA',
    'WIND_SPEED_MAX_PEAK_UPPER_TROPOSPHERE_200HPA',
    'WIND_SPEED_MAX_PEAK_LOWER_STRATOSPHERE_150HPA',
    'WIND_SPEED_MAX_PEAK_MID_STRATOSPHERE_100HPA',
    'WIND_SPEED_MAX_PEAK_UPPER_STRATOSPHERE_50HPA',
    'WIND_SPEED_MAX_PEAK_TOP_STRATOSPHERE_10HPA'
]

# Filter for successful and weather scrubbed launches
successful_launches = launch_corpus[launch_corpus['LAUNCHED'] == 1].copy()
wx_scrub_launches = launch_corpus[launch_corpus['WX_SCRUB'] == 1].copy()

print("Maximum observed wind speed by layer and hour before launch:")
for layer in wind_speed_layers:
    layer_name = layer.replace('WIND_SPEED_MAX_PEAK_', '').replace('_', ' ').title()
    
    # Successful launches
    success_t2 = successful_launches[layer].apply(lambda x: x[-3]).max()
    success_t1 = successful_launches[layer].apply(lambda x: x[-2]).max()
    success_t0 = successful_launches[layer].apply(lambda x: x[-1]).max()
    
    # Weather scrubbed launches
    scrub_t2 = wx_scrub_launches[layer].apply(lambda x: x[-3]).max()
    scrub_t1 = wx_scrub_launches[layer].apply(lambda x: x[-2]).max() 
    scrub_t0 = wx_scrub_launches[layer].apply(lambda x: x[-1]).max()
    
    print(f"\n{layer_name}:")
    print("Successful launches:")
    print(f"T-2: {success_t2:.3f}")
    print(f"T-1: {success_t1:.3f}")
    print(f"T-0: {success_t0:.3f}")
    print("Weather scrubbed launches:")
    print(f"T-2: {scrub_t2:.3f}")
    print(f"T-1: {scrub_t1:.3f}")
    print(f"T-0: {scrub_t0:.3f}")

Maximum observed wind speed by layer and hour before launch:

Boundary Layer 900Hpa:
Successful launches:
T-2: 59.055
T-1: 55.924
T-0: 50.779
Weather scrubbed launches:
T-2: 68.227
T-1: 68.003
T-0: 62.411

Low Troposphere 700Hpa:
Successful launches:
T-2: 76.280
T-1: 71.358
T-0: 74.490
Weather scrubbed launches:
T-2: 76.056
T-1: 88.135
T-0: 100.886

Mid Low Troposphere 500Hpa:
Successful launches:
T-2: 93.280
T-1: 94.623
T-0: 95.965
Weather scrubbed launches:
T-2: 103.794
T-1: 105.807
T-0: 110.729

Mid Troposphere 400Hpa:
Successful launches:
T-2: 112.742
T-1: 112.071
T-0: 111.847
Weather scrubbed launches:
T-2: 141.151
T-1: 140.704
T-0: 132.427

High Mid Troposphere 300Hpa:
Successful launches:
T-2: 131.979
T-1: 130.414
T-0: 131.085
Weather scrubbed launches:
T-2: 148.085
T-1: 144.730
T-0: 144.730

Upper Troposphere 200Hpa:
Successful launches:
T-2: 155.467
T-1: 152.112
T-0: 149.428
Weather scrubbed launches:
T-2: 157.481
T-1: 156.809
T-0: 155.020

Lower Stratosphere 150Hpa:
Successfu

In [106]:
print(launch_corpus[launch_corpus['LAUNCHED'] == 1].head(100)[['START_LCC_EVAL', 'END_LCC_EVAL', 'RAIN', 'WEATHER_CODE']].to_markdown())



|     | START_LCC_EVAL      | END_LCC_EVAL        | RAIN                                                   | WEATHER_CODE                                     |
|----:|:--------------------|:--------------------|:-------------------------------------------------------|:-------------------------------------------------|
|   0 | 2018-01-07 22:00:00 | 2018-01-08 01:00:00 | [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]               | ['1', '2', '0', '1', '1', '2', '1', '3']         |
|   2 | 2018-01-19 18:48:00 | 2018-01-20 00:48:00 | [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]               | ['0', '0', '0', '1', '1', '1', '1', '1']         |
|   4 | 2018-01-31 18:30:00 | 2018-01-31 21:25:00 | [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]               | ['3', '3', '3', '3', '3', '2', '2', '2']         |
|   5 | 2018-02-06 15:30:00 | 2018-02-06 20:45:00 | [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]               | ['1', '0', '0', '0', '1', '2', '2', '0']         |
|   6 | 2018-03-01 16:02:00 | 2018-03-01

In [108]:
print(launch_corpus[launch_corpus['WX_SCRUB'] == 1].head(100)[['START_LCC_EVAL', 'END_LCC_EVAL', 'REMARKS', 'RAIN', 'WEATHER_CODE']].to_markdown())



|     | START_LCC_EVAL      | END_LCC_EVAL        | REMARKS                                                       | RAIN                                                     | WEATHER_CODE                                     |
|----:|:--------------------|:--------------------|:--------------------------------------------------------------|:---------------------------------------------------------|:-------------------------------------------------|
|  26 | 2018-12-20 12:03:00 | 2018-12-20 13:20:00 | Wx Scrub, multiple violations                                 | [0.055, 0.012, 0.043, 0.031, 0.031, 0.028, 0.039, 0.031] | ['61', '51', '55', '53', '53', '53', '55', '53'] |
|  27 | 2018-12-22 11:55:00 | 2018-12-22 14:20:00 | Upper Level Winds                                             | [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                 | ['0', '0', '0', '0', '0', '0', '0', '3']         |
|  32 | 2019-04-10 20:35:00 | 2019-04-10 22:24:00 | Upper Level Winds                           

In [110]:
# Filter for rows where any weather code in the list is above 60
high_weather_code = launch_corpus[launch_corpus['WEATHER_CODE'].apply(lambda x: any(int(code) > 60 for code in x))]
print("\nLaunches with Weather Code > 60:")
print(high_weather_code[['DATE', 'WEATHER_CODE', 'REMARKS', 'WX_SCRUB', 'LAUNCHED']].to_markdown())



Launches with Weather Code > 60:
|     | DATE                | WEATHER_CODE                                     | REMARKS                                                                                            |   WX_SCRUB |   LAUNCHED |
|----:|:--------------------|:-------------------------------------------------|:---------------------------------------------------------------------------------------------------|-----------:|-----------:|
|  20 | 2018-09-10 00:00:00 | ['51', '61', '63', '2', '1', '51', '1', '1']     | NONE                                                                                               |          0 |          1 |
|  26 | 2018-12-20 00:00:00 | ['61', '51', '55', '53', '53', '53', '55', '53'] | Wx Scrub, multiple violations                                                                      |          1 |          0 |
|  32 | 2019-04-10 00:00:00 | ['1', '53', '55', '51', '61', '61', '51', '51']  | Upper Level Winds                                    

Generates a launch report visualization for a specified date, highlighting weather and launch conditions with flexible subplots, labeled axes, status indicators, and formatted time labels. It also includes color-coded statuses, launch window highlighting, and threshold lines for conditions like wind speed and shear, providing a weather and status overview for launch decisions.

In [ ]:
def plot_launch_report(launch_corpus, target_date):
    """
    Creates a comprehensive visualization report for a specific launch date with enhanced formatting
    and visual elements.

    Args:
        launch_corpus (pd.DataFrame): The main dataframe containing all launch data
        target_date: The date to analyze (can be string 'YYYY-MM-DD' or datetime.date)
    """
    # Convert target_date to datetime.date if it's a string
    if isinstance(target_date, str):
        target_date = pd.to_datetime(target_date).date()

    # Get launch data for target date - match on just the date portion of DATETIME
    launch_data = launch_corpus[launch_corpus['DATE'].dt.date == target_date].iloc[0]

    # Create figure with GridSpec for flexible subplot layout
    fig = plt.figure(figsize=(15, 20))
    gs = fig.add_gridspec(8, 2, hspace=1.0, wspace=0.8)  # Increased spacing between columns

    # Enhanced title formatting
    fig.text(0.5, 0.98, f"Launch Report for {target_date}", 
             fontsize=16, weight='bold', ha='center')
    fig.text(0.5, 0.96, f"Vehicle: {launch_data['LAUNCH_VEHICLE']}", 
             fontsize=12, ha='center')

    # Status with color coding
    status_text = 'Weather Scrub' if launch_data['WX_SCRUB'] else 'Weather Delay' if launch_data['WX DELAY'] else 'Nominal'
    status_color = 'red' if launch_data['WX_SCRUB'] else 'orange' if launch_data['WX DELAY'] else 'green'
    fig.text(0.5, 0.95, f"Status: {status_text}", 
             fontsize=12, ha='center', color=status_color)

    # Remarks in italics
    fig.text(0.5, 0.94, f"Remarks: {launch_data['REMARKS']}", 
             fontsize=10, style='italic', ha='center')

    def add_launch_window_highlight(ax):
        """Helper function to add launch window highlighting"""
        start_time = pd.to_datetime(launch_data['START_LCC_EVAL'])
        end_time = pd.to_datetime(launch_data['END_LCC_EVAL'])

        # Convert times to x-axis positions
        start_idx = 7 - (end_time - start_time).seconds // 3600
        end_idx = 7

        # Add transparent highlight
        ylims = ax.get_ylim()
        ax.axvspan(start_idx, end_idx, color='yellow', alpha=0.2, label='Launch Window')
        ax.set_ylim(ylims)  # Restore y-limits after adding span

    def setup_axis(ax, title, ylabel=None):
        """Helper function to setup consistent axis formatting"""
        ax.set_title(title, pad=10, fontsize=10, weight='bold')
        if ylabel:
            ax.set_ylabel(ylabel)
        add_launch_window_highlight(ax)
        # Add left padding for y-axis labels
        ax.yaxis.set_label_coords(-0.15, 0.5)

    # Plot weather parameters
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(range(8), launch_data['TEMPERATURE'], marker='o')
    setup_axis(ax1, 'Temperature', '°F')
    ax1.set_ylim(0, 100)
    ax1.grid(True)

    ax2 = fig.add_subplot(gs[0, 1])
    ax2.plot(range(8), launch_data['CLOUD_COVER'], marker='o')
    setup_axis(ax2, 'Cloud Cover', '%')
    ax2.set_ylim(0, 100)
    ax2.grid(True)

    # Wind speeds at different heights
    ax3 = fig.add_subplot(gs[1, 0])
    ax3.plot(range(8), launch_data['WIND_SPEED_10M'], label='10m', marker='o')
    ax3.plot(range(8), launch_data['WIND_SPEED_100M'], label='100m', marker='o')
    setup_axis(ax3, 'Surface Wind Speed', 'MP/H')
    ax3.set_ylim(0, 30)
    ax3.legend(bbox_to_anchor=(1.12, 1))
    ax3.grid(True)

    # Field Mill readings
    ax4 = fig.add_subplot(gs[1, 1])
    ax4.plot(range(8), launch_data['FIELD_MILL_MAX'], label='Max', marker='o')
    ax4.plot(range(8), launch_data['FIELD_MILL_MEAN'], label='Mean', marker='o')
    ax4.axhline(y=1500, color='r', linestyle='--', label='Critical (1500)')
    ax4.axhline(y=500, color='y', linestyle='--', label='Warning (500)')
    setup_axis(ax4, 'Field Mill Readings')
    # Set y-axis limit based on max value with padding
    max_value = max(max(launch_data['FIELD_MILL_MAX']), 1500)  # At least show critical line
    y_max = max_value * 1.2  # Add 20% padding above max value
    ax4.set_ylim(0, y_max)
    ax4.legend(bbox_to_anchor=(1.12, 1))
    ax4.grid(True)

    # Wind profiles for each atmospheric layer
    layers = ['BOUNDARY_LAYER', 'LOW_TROPOSPHERE', 'MID_TROPOSPHERE', 
              'UPPER_TROPOSPHERE', 'STRATOSPHERE']

    # Speed plots
    ax5 = fig.add_subplot(gs[2:4, 0])
    max_speed = 0
    for layer in layers:
        speed_data = launch_data[f'WIND_SPEED_MAX_PEAK_{layer}']
        max_speed = max(max_speed, max(speed_data))
        ax5.plot(range(8), speed_data, 
                label=layer.replace('_', ' ').title(), marker='o')
    setup_axis(ax5, 'Wind Speed by Atmospheric Layer', 'MP/H')
    ax5.set_ylim(0, max_speed * 1.2)  # Add 20% padding above max value
    ax5.legend(bbox_to_anchor=(1.12, 1))
    ax5.grid(True)

    # Shear plots
    ax6 = fig.add_subplot(gs[2:4, 1])
    max_shear = 0
    for layer in layers:
        shear_data = launch_data[f'WIND_SHEAR_MAX_PEAK_{layer}']
        max_shear = max(max_shear, max(shear_data))
        ax6.plot(range(8), shear_data, 
                label=layer.replace('_', ' ').title(), marker='o')
    setup_axis(ax6, 'Wind Shear by Atmospheric Layer')
    ax6.set_ylim(0, max_shear * 1.2)  # Add 20% padding above max value

    # Add unacceptable threshold lines
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']  # Standard matplotlib colors
    ax6.axhline(y=0.08, color=colors[0], linestyle=':', label='BL Unacceptable (>0.08)', linewidth=0.8)
    ax6.axhline(y=0.09, color=colors[1], linestyle=':', label='LT Unacceptable (>0.09)', linewidth=0.8)
    ax6.axhline(y=0.12, color=colors[2], linestyle=':', label='MT Unacceptable (>0.12)', linewidth=0.8)
    ax6.axhline(y=0.14, color=colors[3], linestyle=':', label='UT Unacceptable (>0.14)', linewidth=0.8)
    ax6.axhline(y=0.16, color=colors[4], linestyle=':', label='S Unacceptable (>0.16)', linewidth=0.8)
    ax6.set_ylim(0, max(max_shear * 1.2, 0.18))  # Ensure threshold lines are visible
    ax6.legend(bbox_to_anchor=(1.12, 1), fontsize='x-small')
    ax6.grid(True)

    # Additional parameters
    ax7 = fig.add_subplot(gs[4, 0])
    ax7.plot(range(8), launch_data['PRESSURE'], marker='o')
    setup_axis(ax7, 'Pressure', 'hPa')
    ax7.set_ylim(900, 1100)
    ax7.grid(True)

    ax8 = fig.add_subplot(gs[4, 1])
    ax8.plot(range(8), launch_data['HUMIDITY'], marker='o')
    setup_axis(ax8, 'Humidity', '%')
    ax8.set_ylim(0, 100)
    ax8.grid(True)

    # Format x-axes for all subplots
    eval_end = pd.to_datetime(launch_data['END_LCC_EVAL'])
    time_labels = [(eval_end - pd.Timedelta(hours=x)).strftime('%H:%M') for x in range(7, -1, -1)]

    for ax in [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8]:
        ax.set_xticks(range(8))
        ax.set_xticklabels(time_labels, rotation=45)
        ax.set_xlabel('Time (GMT+0)')
        ax.grid(True, alpha=0.3)  # Lighter grid

    plt.tight_layout(rect=[0, 0.03, 1, 0.92])
    return fig

# Usage:
fig = plot_launch_report(launch_corpus, '2019-07-24')  # Field Mill Scrub: '2019-07-24', Wind Scrub: 2018-12-22, Cloud Scrub: '2022-01-28'
plt.show()

In [ ]:
# Plot wind speed density and violin plots for each atmospheric layer

# Define atmospheric layers
layers = ['BOUNDARY_LAYER', 'LOW_TROPOSPHERE', 'MID_TROPOSPHERE', 
          'UPPER_TROPOSPHERE', 'STRATOSPHERE']

# Create figure with subplots - 2 columns, one for density plots and one for violin plots
fig = plt.figure(figsize=(24, 20))
gs = gridspec.GridSpec(len(layers), 2, width_ratios=[1, 1.5])

# Find global max wind speed across all layers and launches
max_wind_speed = 0
for layer in layers:
    wind_speeds = launch_corpus[f'WIND_SPEED_MAX_PEAK_{layer}']
    layer_max = max(max(wind_speeds))
    max_wind_speed = max(max_wind_speed, layer_max)

# Plot wind speed density and violin plots for each layer
for layer_idx, layer in enumerate(layers):
    # Density plot
    ax_density = fig.add_subplot(gs[layer_idx, 0])

    # Separate wind issue and no wind issue data for density
    wind_issue_data = []
    no_wind_issue_data = []

    for idx, row in launch_corpus.iterrows():
        data = row[f'WIND_SPEED_MAX_PEAK_{layer}']
        if row['REMARKS_CATEGORY'] == 'wind':
            wind_issue_data.extend(data)
        else:
            no_wind_issue_data.extend(data)

    # Create density plots
    if wind_issue_data:
        sns.kdeplot(data=wind_issue_data, ax=ax_density, color='red', label='Wind Issue', fill=True, alpha=0.3)
    if no_wind_issue_data:
        sns.kdeplot(data=no_wind_issue_data, ax=ax_density, color='blue', label='No Wind Issue', fill=True, alpha=0.3)

    # Set density plot formatting
    ax_density.set_title(f'{layer.replace("_", " ").title()} - Distribution')
    ax_density.set_xlabel('Wind Speed (mph)')
    ax_density.set_ylabel('Density')
    ax_density.grid(True)
    ax_density.legend()
    ax_density.set_xlim(0, max_wind_speed * 1.1)

    # Violin plot
    ax_violin = fig.add_subplot(gs[layer_idx, 1])

    # Prepare data for violin plot
    violin_data = []
    violin_categories = []
    violin_times = []

    for idx, row in launch_corpus.iterrows():
        data = row[f'WIND_SPEED_MAX_PEAK_{layer}']
        category = 'Wind Issue' if row['REMARKS_CATEGORY'] == 'wind' else 'No Wind Issue'
        for hour, value in enumerate(data):
            violin_data.append(value)
            violin_categories.append(category)
            violin_times.append(f'T-{7-hour}')

    # Create violin plot
    violin_df = pd.DataFrame({
        'Wind Speed': violin_data,
        'Category': violin_categories,
        'Time': violin_times
    })

    sns.violinplot(data=violin_df, x='Time', y='Wind Speed', hue='Category',
                  ax=ax_violin, split=True, inner='quartile',
                  palette={'Wind Issue': 'red', 'No Wind Issue': 'blue'})

    # Set violin plot formatting
    ax_violin.set_title(f'{layer.replace("_", " ").title()} - Time Series Distribution')
    ax_violin.set_xlabel('Hours Before Launch')
    ax_violin.set_ylabel('Wind Speed (mph)')
    ax_violin.grid(True)
    ax_violin.set_ylim(0, max_wind_speed * 1.1)

plt.tight_layout()
plt.show()

| Atmospheric Layer | Wind Speed Threshold | Key Observations |
|------------------|---------------------|------------------|
| Boundary Layer | 25-30 MPH | - Sharp cutoff in safe launches beyond 30 MPH<br>- Most restrictive layer overall<br>- Very clear distinction between safe/unsafe distributions |
| Low Troposphere | 50-75 MPH | - Wider tolerance than boundary layer<br>- Gradual increase in issues rather than sharp cutoff<br>- Shows bimodal distribution in wind issues |
| Mid Troposphere | 75-100 MPH | - Significantly higher tolerance<br>- Broad distribution of safe launches<br>- Wind issues spread across wider range |
| Upper Troposphere | 100-125 MPH | - Highest wind speed tolerance of all layers<br>- Very wide distribution<br>- Notable overlap between safe/unsafe regions |
| Stratosphere | 40-50 MPH | - More restrictive than middle layers<br>- Tighter distribution overall<br>- Clear peak in safe launch window |

In [ ]:
# Plot field mill density and violin plots

# Create figure with subplots - 2 columns, one for density plots and one for violin plots
fig = plt.figure(figsize=(24, 10))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1.5])

# Find global max field mill value
max_field_mill = max(max(row['FIELD_MILL_MAX']) for _, row in launch_corpus.iterrows())

# Density plot
ax_density = fig.add_subplot(gs[0, 0])

# Separate lightning issue and no lightning issue data for density
lightning_issue_data = []
no_lightning_issue_data = []

for idx, row in launch_corpus.iterrows():
    data = row['FIELD_MILL_MAX']
    if row['REMARKS_CATEGORY'] == 'lightning':
        lightning_issue_data.extend(data)
    else:
        no_lightning_issue_data.extend(data)

# Create density plots
if lightning_issue_data:
    sns.kdeplot(data=lightning_issue_data, ax=ax_density, color='red', label='Lightning Issue', fill=True, alpha=0.3)
if no_lightning_issue_data:
    sns.kdeplot(data=no_lightning_issue_data, ax=ax_density, color='blue', label='No Lightning Issue', fill=True, alpha=0.3)

# Set density plot formatting
ax_density.set_title('Field Mill Readings - Distribution')
ax_density.set_xlabel('Field Mill Reading (V/m)')
ax_density.set_ylabel('Density')
ax_density.grid(True)
ax_density.legend()
ax_density.set_xlim(0, max_field_mill * 1.1)

# x-axis ticks
ax_density.xaxis.set_major_locator(plt.MaxNLocator(20))

# Violin plot
ax_violin = fig.add_subplot(gs[0, 1])

# Prepare data for violin plot
violin_data = []
violin_categories = []
violin_times = []

for idx, row in launch_corpus.iterrows():
    data = row['FIELD_MILL_MAX']
    category = 'Lightning Issue' if row['REMARKS_CATEGORY'] == 'lightning' else 'No Lightning Issue'
    for hour, value in enumerate(data):
        violin_data.append(value)
        violin_categories.append(category)
        violin_times.append(f'T-{7-hour}')

# Create violin plot
violin_df = pd.DataFrame({
    'Field Mill': violin_data,
    'Category': violin_categories,
    'Time': violin_times
})

sns.violinplot(data=violin_df, x='Time', y='Field Mill', hue='Category',
              ax=ax_violin, split=True, inner='quartile',
              palette={'Lightning Issue': 'red', 'No Lightning Issue': 'blue'})

# Set violin plot formatting
ax_violin.set_title('Field Mill Readings - Time Series Distribution')
ax_violin.set_xlabel('Hours Before Launch')
ax_violin.set_ylabel('Field Mill Reading (V/m)')
ax_violin.grid(True)
ax_violin.set_ylim(0, max_field_mill * 1.1)

# Add more y-axis ticks
ax_violin.yaxis.set_major_locator(plt.MaxNLocator(20))

plt.tight_layout()
plt.show()


| Field Mill Reading Range | Risk Assessment | Observations |
|-------------------------|-----------------|--------------|
| 0-600 V/m | Generally Safe | - Majority of successful launches<br>- Optimal range for launch<br>- High confidence zone |
| 600-3000 V/m | Increased Risk | - Transition zone<br>- Requires careful monitoring<br>- May indicate developing conditions |
| 3000+ V/m | High Risk | - Strong indicator of lightning risk<br>- Rare in successful launches<br>- Likely launch scrub condition |

In [ ]:
# Plot cloud cover density and violin plots

# Create figure with subplots - 2 columns, one for density plots and one for violin plots
fig = plt.figure(figsize=(24, 10))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1.5])

# Find global max cloud cover value
max_cloud_cover = max(max(row['CLOUD_COVER']) for _, row in launch_corpus.iterrows())

# Density plot
ax_density = fig.add_subplot(gs[0, 0])

# Separate cloud issue and no cloud issue data for density
cloud_issue_data = []
no_cloud_issue_data = []

for idx, row in launch_corpus.iterrows():
    data = row['CLOUD_COVER']
    if row['REMARKS_CATEGORY'] == 'clouds':
        cloud_issue_data.extend(data)
    else:
        no_cloud_issue_data.extend(data)

# Create density plots
if cloud_issue_data:
    sns.kdeplot(data=cloud_issue_data, ax=ax_density, color='red', label='Cloud Issue', fill=True, alpha=0.3)
if no_cloud_issue_data:
    sns.kdeplot(data=no_cloud_issue_data, ax=ax_density, color='blue', label='No Cloud Issue', fill=True, alpha=0.3)

# Set density plot formatting
ax_density.set_title('Cloud Cover - Distribution')
ax_density.set_xlabel('Cloud Cover (%)')
ax_density.set_ylabel('Density')
ax_density.grid(True)
ax_density.legend()
ax_density.set_xlim(0, max_cloud_cover * 1.1)

# x-axis ticks
ax_density.xaxis.set_major_locator(plt.MaxNLocator(20))

# Violin plot
ax_violin = fig.add_subplot(gs[0, 1])

# Prepare data for violin plot
violin_data = []
violin_categories = []
violin_times = []

for idx, row in launch_corpus.iterrows():
    data = row['CLOUD_COVER']
    category = 'Cloud Issue' if row['REMARKS_CATEGORY'] == 'clouds' else 'No Cloud Issue'
    for hour, value in enumerate(data):
        violin_data.append(value)
        violin_categories.append(category)
        violin_times.append(f'T-{7-hour}')

# Create violin plot
violin_df = pd.DataFrame({
    'Cloud Cover': violin_data,
    'Category': violin_categories,
    'Time': violin_times
})

sns.violinplot(data=violin_df, x='Time', y='Cloud Cover', hue='Category',
              ax=ax_violin, split=True, inner='quartile',
              palette={'Cloud Issue': 'red', 'No Cloud Issue': 'blue'})

# Set violin plot formatting
ax_violin.set_title('Cloud Cover - Time Series Distribution')
ax_violin.set_xlabel('Hours Before Launch')
ax_violin.set_ylabel('Cloud Cover (%)')
ax_violin.grid(True)
ax_violin.set_ylim(0, max_cloud_cover * 1.1)

# Add more y-axis ticks
ax_violin.yaxis.set_major_locator(plt.MaxNLocator(20))

plt.tight_layout()
plt.show()


| Cloud Cover Range | Risk Assessment | Observations |
|------------------|-----------------|--------------|
| 0-15% | Optimal | - First safe zone peak<br>- Highly favorable for launch<br>- Clearest viewing conditions |
| 15-35% | Generally Safe | - Second safe zone peak<br>- Still favorable conditions<br>- Acceptable launch window |
| 35-50% | Increased Risk | - Transition zone<br>- Beginning of cloud issues<br>- Requires careful evaluation |
| >50% | High Risk | - Dominated by cloud issues<br>- Secondary risk peak at 85-90%<br>- Likely launch constraint |

### Prepare train/test set

In [ ]:
print(launch_corpus.sample(10).to_markdown())

Define target column

In [77]:
# 1 means no-go (scrubbed), 0 means go (not scrubbed)
launch_corpus['NOGO_FLAG'] = launch_corpus['WX_SCRUB'].astype(int)

Select features

In [78]:
# Assemble features
temp_features = ['TEMPERATURE']
precip_features = ['RAIN']
cloud_features = ['CLOUD_COVER']
field_mill_features = ['FIELD_MILL_MAX']
wind_speed_features = ['WIND_SPEED_10M', 'WIND_SPEED_100M', 'WIND_SPEED_MAX_PEAK_BOUNDARY_LAYER', 'WIND_SPEED_MAX_PEAK_LOW_TROPOSPHERE', 'WIND_SPEED_MAX_PEAK_MID_TROPOSPHERE', 'WIND_SPEED_MAX_PEAK_UPPER_TROPOSPHERE', 'WIND_SPEED_MAX_PEAK_STRATOSPHERE']
wind_shear_features = ['WIND_SHEAR_MAX_PEAK_BOUNDARY_LAYER', 'WIND_SHEAR_MAX_PEAK_LOW_TROPOSPHERE', 'WIND_SHEAR_MAX_PEAK_MID_TROPOSPHERE', 'WIND_SHEAR_MAX_PEAK_UPPER_TROPOSPHERE', 'WIND_SHEAR_MAX_PEAK_STRATOSPHERE']
weather_code_features = ['WEATHER_CODE']
pressure_features = ['PRESSURE']
humidity_features = ['HUMIDITY']

# Combine features
features = temp_features + precip_features + cloud_features + field_mill_features + wind_speed_features + wind_shear_features + weather_code_features + pressure_features + humidity_features

Ensure selected features are consistent in time steps

In [ ]:
# Check for any missing values in the features
missing_values = launch_corpus[features].isnull().sum()
print("\nMissing values in features:")
print(missing_values[missing_values > 0])

# Check for infinite values
# First convert numeric columns to float to handle inf values
numeric_features = launch_corpus[features].select_dtypes(include=['int64', 'float64']).columns
infinite_values = pd.Series(0, index=features)  # Initialize with zeros for all features

if len(numeric_features) > 0:
    infinite_values[numeric_features] = np.isinf(launch_corpus[numeric_features].replace([np.inf, -np.inf], np.nan)).sum()

print("\nInfinite values in features:")
print(infinite_values[infinite_values > 0])

Scan launch_corpus for array-like entries with NaNs, convert infinities to NaNs, fill NaNs in arrays using edge fill and middle interpolation, log rows with filled NaNs, drop features if entirely NaN, ensure no NaNs remain, and record the new dataset size.

In [ ]:
def handle_array_nans_in_features(launch_corpus, features):
    """
    Handle NaN values in array-like features by:
    1. Checking for NaN values
    2. Converting infinities to NaNs
    3. Filling NaNs using interpolation and edge filling
    4. Dropping rows with all NaN features
    5. Verifying results

    Args:
        launch_corpus: DataFrame containing the launch data
        features: List of feature column names to process

    Returns:
        Processed DataFrame with NaNs handled
    """
    # 1. First check for NaN values
    array_nan_data = []
    for idx, row in launch_corpus[features].iterrows():
        nan_features = {}
        for feature in features:
            value = row[feature]
            if isinstance(value, (list, np.ndarray)):
                if any(pd.isna(x) for x in value):
                    nan_count = sum(pd.isna(x) for x in value)
                    nan_features[feature] = f"{nan_count} NaN values"
        if nan_features:
            nan_features['Row'] = idx
            array_nan_data.append(nan_features)

    if array_nan_data:
        print("\nInitial NaN values in array features:")
        print(pd.DataFrame(array_nan_data).set_index('Row').to_markdown())
    else:
        print("\nNo initial NaN values found in array features")

    # 2. Replace infinities with NaN
    def replace_inf_with_nan(arr):
        if isinstance(arr, (list, np.ndarray)):
            arr = np.array(arr, dtype=float)
            arr[np.isinf(arr)] = np.nan
            return arr
        return arr

    for feature in features:
        launch_corpus[feature] = launch_corpus[feature].apply(replace_inf_with_nan)

    # 3. Fill NaN values
    def fill_array_nans(arr):
        if not isinstance(arr, (list, np.ndarray)):
            return arr

        try:
            arr = np.array(arr, dtype=float)
        except (ValueError, TypeError):
            return arr

        if np.all(np.isnan(arr)):
            return None

        series = pd.Series(arr)
        filled = series.interpolate(method='linear')
        filled = filled.fillna(method='ffill').fillna(method='bfill')
        return filled.values

    initial_length = len(launch_corpus)

    for feature in features:
        launch_corpus[feature] = launch_corpus[feature].apply(fill_array_nans)

    # 4. Drop rows where any feature returned None
    launch_corpus = launch_corpus.dropna(subset=features)

    # 5. Verify results
    array_nan_data = []
    for idx, row in launch_corpus[features].iterrows():
        nan_features = {}
        for feature in features:
            value = row[feature]
            if isinstance(value, (list, np.ndarray)):
                if any(pd.isna(x) for x in value):
                    nan_count = sum(pd.isna(x) for x in value)
                    nan_features[feature] = f"{nan_count} NaN values"
        if nan_features:
            nan_features['Row'] = idx
            array_nan_data.append(nan_features)

    if array_nan_data:
        print("\nRemaining NaN values after filling:")
        print(pd.DataFrame(array_nan_data).set_index('Row').to_markdown())
    else:
        print("\nAll NaN values have been successfully handled")

    print(f"\nDataset length: {len(launch_corpus)} rows (removed {initial_length - len(launch_corpus)} rows)")

    return launch_corpus

launch_corpus = handle_array_nans_in_features(launch_corpus, features)

#### Categorical Balancing

Filter weather dataset to include only data from 2018 onwards and rename specific columns.

In [81]:
# Rename columns to simplified names and filter for dates after 2018
weather_filtered = weather[weather['TIME'].dt.year > 2017].rename(columns={
    'TEMPERATURE_2M (°F)': 'TEMPERATURE',
    'PRECIPITATION (INCH)': 'PRECIPITATION', 
    'RAIN (INCH)': 'RAIN',
    'CLOUD_COVER (%)': 'CLOUD_COVER',
    'WEATHER_CODE (WMO CODE)': 'WEATHER_CODE',
    'PRESSURE_MSL (HPA)': 'PRESSURE',
    'RELATIVE_HUMIDITY_2M (%)': 'HUMIDITY',
    'WIND_SPEED_10M (MP/H)': 'WIND_SPEED_10M',
    'WIND_SPEED_100M (MP/H)': 'WIND_SPEED_100M'
})
weather_filtered = weather_filtered[['TIME', 'TEMPERATURE', 'RAIN', 'CLOUD_COVER', 'WEATHER_CODE', 'PRESSURE', 'HUMIDITY', 'WIND_SPEED_10M', 'WIND_SPEED_100M']]

Aggregates hourly maximum values from multiple field mill sensors into a single maximum value per hour.

In [82]:
# Aggregate field mill data by hour, taking max across sensors
field_mill_agg = field_mill.groupby('DATETIME')['MAX'].max().reset_index().rename(columns={'MAX': 'FIELD_MILL_MAX'})

Transforms wind profile data by pivoting it to create separate columns for each atmospheric layer, focusing on maximum wind speed and shear, then flattens the resulting multi-index column names.

In [83]:
# Pivot wind profiles to get layer-specific columns
wind_profiles_wide = binned_wind_profiles.pivot(
    index='DATETIME',
    columns='ATMOSPHERIC_LAYER',
    values=['SPEED_MAX_PEAK', 'SHEAR_MAX_PEAK']
).reset_index()

# Flatten column names
wind_profiles_wide.columns = [
    f'WIND_{col[0]}_{col[1].upper()}'.replace(' ', '_') 
    if col[1] else 'DATETIME' 
    for col in wind_profiles_wide.columns
]

Merge weather data, field mill readings, and wind profiles into a unified dataframe based on datetime, ensuring all numerical columns are converted to float.

In [84]:
def merge_weather_data(weather_df, field_mill_df, wind_profiles_df):
    """
    Merge weather, field mill, and wind profile data into a single dataframe.

    Args:
        weather_df: Weather dataframe with hourly measurements
        field_mill_df: Aggregated field mill readings
        wind_profiles_df: Wide-format wind profile data

    Returns:
        DataFrame with all weather measurements merged on datetime
    """
    # Merge all dataframes on TIME/DATETIME columns
    merged_data = (weather_df
                  .merge(field_mill_df, left_on='TIME', right_on='DATETIME')
                  .drop(columns=['DATETIME'])
                  .merge(wind_profiles_df, left_on='TIME', right_on='DATETIME', how='left')
                  .drop(columns=['DATETIME']))

    # Convert all columns except TIME and WEATHER_CODE to float
    for col in merged_data.columns:
        if col not in ['TIME', 'WEATHER_CODE']:
            merged_data[col] = merged_data[col].astype(float)

    return merged_data

all_weather_data = merge_weather_data(weather_filtered, field_mill_agg, wind_profiles_wide)

Define a WeatherViolationChecker class that checks for weather-related violations (like lightning, wind, and cloud cover) within specified time windows, analyzes patterns of these violations, and generates synthetic samples of weather data while ensuring balance across different categories and window sizes.

In [ ]:
class WeatherViolationChecker:
    def __init__(self):
        # Thresholds
        self.THRESHOLDS = {
            'field_mill': 600,
            'cloud_cover': 50,
            'wind': {
                'boundary_layer': 30,
                'low_troposphere': 75,
                'mid_troposphere': 100,
                'upper_troposphere': 125,
                'stratosphere': 50
            }
        }

        # Required window sizes
        self.WINDOW_SIZES = [2, 3, 4, 6]

        # Column mappings for wind layers
        self.WIND_COLUMNS = {
            'boundary_layer': 'WIND_SPEED_MAX_PEAK_BOUNDARY_LAYER',
            'low_troposphere': 'WIND_SPEED_MAX_PEAK_LOW_TROPOSPHERE',
            'mid_troposphere': 'WIND_SPEED_MAX_PEAK_MID_TROPOSPHERE',
            'upper_troposphere': 'WIND_SPEED_MAX_PEAK_UPPER_TROPOSPHERE',
            'stratosphere': 'WIND_SPEED_MAX_PEAK_STRATOSPHERE'
        }

    def check_wind_violations(self, row):
        """Check wind violations across all atmospheric layers."""
        violated_layers = []

        for layer, column in self.WIND_COLUMNS.items():
            # Add null check for wind values
            if pd.notna(row[column]) and row[column] > self.THRESHOLDS['wind'][layer]:
                violated_layers.append(layer)

        return violated_layers if len(violated_layers) >= 2 else []

    def analyze_violation_pattern(self, violation_hours, window_size):
        """Analyze if violations are continuous or intermittent."""
        if not violation_hours:
            return {'type': 'none', 'gaps': []}

        sorted_hours = sorted(violation_hours)
        gaps = []

        for i in range(1, len(sorted_hours)):
            gap = (sorted_hours[i] - sorted_hours[i-1]).total_seconds() / 3600
            gaps.append(gap)

        is_continuous = all(gap == 1 for gap in gaps)

        return {
            'type': 'continuous' if is_continuous else 'intermittent',
            'gaps': gaps,
            'duration': (sorted_hours[-1] - sorted_hours[0]).total_seconds() / 3600 + 1 if sorted_hours else 0
        }

    def check_window_violations(self, window_data):
        """
        Comprehensive check for violations in a time window.
        Returns detailed violation information.
        """
        if len(window_data) == 0:
            return None

        violations = {
            'lightning': False,
            'wind': False,
            'cloud': False,
            'details': {
                'lightning_hours': [],
                'wind_layers': {},
                'cloud_hours': [],
                'patterns': {}
            }
        }

        # Check each hour in the window
        for idx, row in window_data.iterrows():
            # Lightning check (with null check)
            if pd.notna(row['FIELD_MILL_MAX']) and row['FIELD_MILL_MAX'] > self.THRESHOLDS['field_mill']:
                violations['lightning'] = True
                violations['details']['lightning_hours'].append(row['TIME'])

            # Wind check
            wind_violations = self.check_wind_violations(row)
            if wind_violations:
                violations['wind'] = True
                violations['details']['wind_layers'][row['TIME']] = wind_violations

            # Cloud cover check (with null check)
            if pd.notna(row['CLOUD_COVER']) and row['CLOUD_COVER'] > self.THRESHOLDS['cloud_cover']:
                violations['cloud'] = True
                violations['details']['cloud_hours'].append(row['TIME'])

        # Verify final hour has violation(s)
        final_time = window_data['TIME'].iloc[-1]
        has_final_violation = any([
            final_time in violations['details']['lightning_hours'],
            final_time in violations['details']['wind_layers'],
            final_time in violations['details']['cloud_hours']
        ])

        if not has_final_violation:
            return None

        # Analyze patterns
        window_size = len(window_data)
        violations['details']['patterns'] = {
            'lightning': self.analyze_violation_pattern(
                violations['details']['lightning_hours'], 
                window_size
            ),
            'wind': self.analyze_violation_pattern(
                list(violations['details']['wind_layers'].keys()),
                window_size
            ),
            'cloud': self.analyze_violation_pattern(
                violations['details']['cloud_hours'],
                window_size
            )
        }

        # Generate category and subcategory
        active_violations = sorted([
            k for k, v in violations.items() 
            if v is True and k != 'details'
        ])

        violations['category'] = '_'.join(active_violations)

        # Add pattern type to subcategory
        pattern_types = []
        for violation_type in active_violations:
            pattern = violations['details']['patterns'][violation_type]['type']
            pattern_types.append(f"{violation_type}_{pattern}")
        violations['subcategory'] = '_'.join(pattern_types)

        return violations

    def generate_synthetic_samples(self, merged_data, exclude_dates=None, 
                                min_samples_per_category=100, 
                                min_samples_per_window=50):
        """
        Generate synthetic samples with balanced categories and window sizes.
        """
        print("\nStarting synthetic sample generation...")
        samples = []
        category_counts = defaultdict(int)
        window_size_counts = defaultdict(int)
        subcategory_counts = defaultdict(int)

        # Convert exclude_dates to set of datetime.date objects
        if exclude_dates is not None:
            print(f"Processing {len(exclude_dates)} excluded dates...")
            if isinstance(next(iter(exclude_dates)), pd.Timestamp):
                exclude_dates = {d.date() for d in exclude_dates}
            else:
                exclude_dates = {pd.to_datetime(d).date() for d in exclude_dates}

        total_rows = len(merged_data)
        print(f"\nProcessing {total_rows} total rows of weather data...")
        progress_interval = max(1, total_rows // 20)  # Show progress every 5%

        for i in range(8, len(merged_data)):
            if i % progress_interval == 0:
                print(f"Progress: {i}/{total_rows} rows ({(i/total_rows*100):.1f}%)")
                print(f"Current samples found: {len(samples)}")

            end_time = merged_data.iloc[i]['TIME']

            if exclude_dates and end_time.date() in exclude_dates:
                continue

            full_window = merged_data.iloc[i-8:i].copy()

            for window_size in self.WINDOW_SIZES:
                window = merged_data.iloc[i-window_size:i].copy()
                violations = self.check_window_violations(window)

                if violations:
                    category = violations['category']
                    subcategory = violations['subcategory']

                    # Update counts
                    category_counts[category] += 1
                    window_size_counts[window_size] += 1
                    subcategory_counts[subcategory] += 1

                    # Create sample with array data
                    sample = {
                        'START_TIME': full_window['TIME'].iloc[0],
                        'END_TIME': end_time,
                        'WINDOW_SIZE': window_size,
                        'CATEGORY': category,
                        'SUBCATEGORY': subcategory,
                        'SYNTHETIC': True,
                        'HAS_LIGHTNING': violations['lightning'],
                        'HAS_WIND': violations['wind'],
                        'HAS_CLOUD': violations['cloud'],
                        'LIGHTNING_PATTERN': violations['details']['patterns']['lightning']['type'],
                        'WIND_PATTERN': violations['details']['patterns']['wind']['type'],
                        'CLOUD_PATTERN': violations['details']['patterns']['cloud']['type']
                    }

                    # Add weather data as arrays
                    for col in merged_data.columns:
                        if col != 'TIME':  # Skip TIME column
                            sample[col] = full_window[col].values

                    samples.append(sample)

                # Check if we have enough samples
                if (all(count >= min_samples_per_category 
                    for count in category_counts.values()) and
                    all(count >= min_samples_per_window 
                        for count in window_size_counts.values())):
                    break

        # Convert to DataFrame
        print("\nConverting samples to DataFrame...")
        samples_df = pd.DataFrame(samples)

        if len(samples_df) > 0:
            # Print summary statistics
            print("\nSynthetic Samples Generated:")
            print(f"Total samples: {len(samples_df)}")
            print("\nSamples per category:")
            print(samples_df['CATEGORY'].value_counts().to_markdown())
            print("\nSamples per window size:")
            print(samples_df['WINDOW_SIZE'].value_counts().to_markdown())
            print("\nSamples per subcategory:")
            print(samples_df['SUBCATEGORY'].value_counts().to_markdown())
        else:
            print("\nNo samples generated!")

        return samples_df

# Usage:
checker = WeatherViolationChecker()
exclude_dates = set(launch_corpus['DATE'].unique())
bad_synthetic_samples = checker.generate_synthetic_samples(
    all_weather_data,
    exclude_dates=exclude_dates,
    min_samples_per_category=100,
    min_samples_per_window=50
)

In [ ]:
print(bad_synthetic_samples.head(2).to_markdown())

Define a GoodWeatherChecker class that evaluates weather conditions to determine if they are favorable across various parameters like lightning activity, wind speeds, and cloud cover within specified time windows. It then generates synthetic samples of good weather conditions, ensuring a balance across different categories and window sizes.

In [ ]:
class GoodWeatherChecker:
    def __init__(self):
        # Thresholds for good conditions
        self.THRESHOLDS = {
            'field_mill': {
                'max': 600,  # Must be below this
                'preferred_max': 300  # Ideal range
            },
            'cloud_cover': {
                'optimal': 15,  # 0-15%
                'acceptable': 35  # 15-35%
            },
            'wind': {
                # Setting conservative thresholds well below the danger zones
                'boundary_layer': 20,  # Safe below 25-30
                'low_troposphere': 45,  # Safe below 50-75
                'mid_troposphere': 65,  # Safe below 75-100
                'upper_troposphere': 90,  # Safe below 100-125
                'stratosphere': 35   # Safe below 40-50
            }
        }

        # Required window sizes
        self.WINDOW_SIZES = [2, 3, 4, 6]

        # Column mappings for wind layers
        self.WIND_COLUMNS = {
            'boundary_layer': 'WIND_SPEED_MAX_PEAK_BOUNDARY_LAYER',
            'low_troposphere': 'WIND_SPEED_MAX_PEAK_LOW_TROPOSPHERE',
            'mid_troposphere': 'WIND_SPEED_MAX_PEAK_MID_TROPOSPHERE',
            'upper_troposphere': 'WIND_SPEED_MAX_PEAK_UPPER_TROPOSPHERE',
            'stratosphere': 'WIND_SPEED_MAX_PEAK_STRATOSPHERE'
        }

    def check_good_wind_conditions(self, row):
        """Check if wind conditions are favorable across all layers."""
        good_layers = []

        for layer, column in self.WIND_COLUMNS.items():
            if pd.notna(row[column]) and row[column] <= self.THRESHOLDS['wind'][layer]:
                good_layers.append(layer)

        # Return layers only if ALL are good (inverse of violation logic)
        return good_layers if len(good_layers) == len(self.WIND_COLUMNS) else []

    def analyze_condition_pattern(self, good_hours, window_size):
        """Analyze if good conditions are continuous or intermittent."""
        if not good_hours:
            return {'type': 'none', 'gaps': []}

        sorted_hours = sorted(good_hours)
        gaps = []

        for i in range(1, len(sorted_hours)):
            gap = (sorted_hours[i] - sorted_hours[i-1]).total_seconds() / 3600
            gaps.append(gap)

        is_continuous = all(gap == 1 for gap in gaps)

        return {
            'type': 'continuous' if is_continuous else 'intermittent',
            'gaps': gaps,
            'duration': (sorted_hours[-1] - sorted_hours[0]).total_seconds() / 3600 + 1 if sorted_hours else 0
        }

    def check_window_conditions(self, window_data):
        """
        Check for good weather conditions in a time window.
        """
        if len(window_data) == 0:
            return None

        conditions = {
            'good_lightning': False,
            'good_wind': False,
            'good_cloud': False,
            'details': {
                'good_lightning_hours': [],
                'good_wind_hours': {},
                'good_cloud_hours': [],
                'patterns': {}
            }
        }

        # Check each hour in the window
        for idx, row in window_data.iterrows():
            # Lightning check (with null check)
            if pd.notna(row['FIELD_MILL_MAX']) and row['FIELD_MILL_MAX'] <= self.THRESHOLDS['field_mill']['max']:
                conditions['good_lightning'] = True
                conditions['details']['good_lightning_hours'].append(row['TIME'])

            # Wind check
            good_wind_layers = self.check_good_wind_conditions(row)
            if good_wind_layers:
                conditions['good_wind'] = True
                conditions['details']['good_wind_hours'][row['TIME']] = good_wind_layers

            # Cloud cover check (with null check)
            if pd.notna(row['CLOUD_COVER']) and row['CLOUD_COVER'] <= self.THRESHOLDS['cloud_cover']['acceptable']:
                conditions['good_cloud'] = True
                conditions['details']['good_cloud_hours'].append(row['TIME'])

        # Verify final hour has good conditions
        final_time = window_data['TIME'].iloc[-1]
        has_final_good = all([
            final_time in conditions['details']['good_lightning_hours'],
            final_time in conditions['details']['good_wind_hours'],
            final_time in conditions['details']['good_cloud_hours']
        ])

        if not has_final_good:
            return None

        # Analyze patterns
        window_size = len(window_data)
        conditions['details']['patterns'] = {
            'lightning': self.analyze_condition_pattern(
                conditions['details']['good_lightning_hours'], 
                window_size
            ),
            'wind': self.analyze_condition_pattern(
                list(conditions['details']['good_wind_hours'].keys()),
                window_size
            ),
            'cloud': self.analyze_condition_pattern(
                conditions['details']['good_cloud_hours'],
                window_size
            )
        }

        # Generate category and subcategory based on good conditions
        active_conditions = sorted([
            k.replace('good_', '') for k, v in conditions.items() 
            if k.startswith('good_') and v is True and k != 'details'
        ])

        conditions['category'] = 'good_' + '_'.join(active_conditions)

        # Add pattern type to subcategory
        pattern_types = []
        for condition_type in active_conditions:
            pattern = conditions['details']['patterns'][condition_type]['type']
            pattern_types.append(f"{condition_type}_{pattern}")
        conditions['subcategory'] = 'good_' + '_'.join(pattern_types)

        return conditions

    def generate_synthetic_samples(self, merged_data, exclude_dates=None, 
                                min_samples_per_category=100, 
                                min_samples_per_window=50):
        """
        Generate synthetic samples with balanced categories and window sizes.
        """
        print("\nStarting good weather synthetic sample generation...")
        samples = []
        category_counts = defaultdict(int)
        window_size_counts = defaultdict(int)
        subcategory_counts = defaultdict(int)

        # Convert exclude_dates to set of datetime.date objects
        if exclude_dates is not None:
            print(f"\nProcessing {len(exclude_dates)} excluded dates...")
            if isinstance(next(iter(exclude_dates)), pd.Timestamp):
                exclude_dates = {d.date() for d in exclude_dates}
            else:
                exclude_dates = {pd.to_datetime(d).date() for d in exclude_dates}

        print(f"\nProcessing {len(merged_data)} total rows of weather data...")
        progress_interval = len(merged_data) // 20  # Show progress every 5%

        for i in range(8, len(merged_data)):
            if i % progress_interval == 0:
                print(f"Progress: {i}/{len(merged_data)} rows ({(i/len(merged_data)*100):.1f}%)")
                print(f"Current samples found: {len(samples)}")

            end_time = merged_data.iloc[i]['TIME']

            if exclude_dates and end_time.date() in exclude_dates:
                continue

            full_window = merged_data.iloc[i-8:i].copy()

            for window_size in self.WINDOW_SIZES:
                window = merged_data.iloc[i-window_size:i].copy()
                conditions = self.check_window_conditions(window)

                if conditions:
                    category = conditions['category']
                    subcategory = conditions['subcategory']

                    # Update counts
                    category_counts[category] += 1
                    window_size_counts[window_size] += 1
                    subcategory_counts[subcategory] += 1

                    # Create sample with array data
                    sample = {
                        'START_TIME': full_window['TIME'].iloc[0],
                        'END_TIME': end_time,
                        'WINDOW_SIZE': window_size,
                        'CATEGORY': category,
                        'SUBCATEGORY': subcategory,
                        'SYNTHETIC': True,
                        'HAS_GOOD_LIGHTNING': conditions['good_lightning'],
                        'HAS_GOOD_WIND': conditions['good_wind'],
                        'HAS_GOOD_CLOUD': conditions['good_cloud'],
                        'LIGHTNING_PATTERN': conditions['details']['patterns']['lightning']['type'],
                        'WIND_PATTERN': conditions['details']['patterns']['wind']['type'],
                        'CLOUD_PATTERN': conditions['details']['patterns']['cloud']['type']
                    }

                    # Add weather data as arrays
                    for col in merged_data.columns:
                        if col != 'TIME':  # Skip TIME column
                            sample[col] = full_window[col].values

                    samples.append(sample)

                # Check if we have enough samples
                if (all(count >= min_samples_per_category 
                    for count in category_counts.values()) and
                    all(count >= min_samples_per_window 
                        for count in window_size_counts.values())):
                    break

        # Convert to DataFrame
        print("\nConverting samples to DataFrame...")
        samples_df = pd.DataFrame(samples)

        if len(samples_df) > 0:
            # Print summary statistics
            print("\nGood Weather Synthetic Samples Generated:")
            print(f"Total samples: {len(samples_df)}")
            print("\nSamples per category:")
            print(samples_df['CATEGORY'].value_counts().to_markdown())
            print("\nSamples per window size:")
            print(samples_df['WINDOW_SIZE'].value_counts().to_markdown())
            print("\nSamples per subcategory:")
            print(samples_df['SUBCATEGORY'].value_counts().to_markdown())
        else:
            print("\nNo samples generated!")

        return samples_df

# Usage:
checker = GoodWeatherChecker()
exclude_dates = set(launch_corpus['DATE'].unique())
good_synthetic_samples = checker.generate_synthetic_samples(
   all_weather_data,
   exclude_dates=exclude_dates,
   min_samples_per_category=100,
   min_samples_per_window=50
)

In [ ]:
# Handle array nans for both synthetic datasets
bad_synthetic_samples = handle_array_nans_in_features(bad_synthetic_samples, features)
good_synthetic_samples = handle_array_nans_in_features(good_synthetic_samples, features)

In [ ]:
# Filter bad weather samples to exclude cloud-only violations and balance with good weather samples.
def filter_bad_weather_samples(bad_samples):
    """
    Filter bad weather samples to exclude cloud-only violations and balance with good weather samples.
    Returns filtered dataframe with selected categories.
    """
    # Get counts of good weather samples for reference
    good_weather_count = len(good_synthetic_samples)
    print(f"Good weather samples available: {good_weather_count}")

    # List categories we want to keep (excluding cloud-only)
    desired_categories = [
        'wind',
        'cloud_lightning',
        'cloud_wind', 
        'lightning',
        'cloud_lightning_wind',
        'lightning_wind'
    ]

    # Filter for desired categories
    filtered_bad = bad_samples[bad_samples['CATEGORY'].isin(desired_categories)]

    # Show distribution of categories
    print("\nBad weather category distribution before sampling:")
    print(filtered_bad['CATEGORY'].value_counts().to_markdown())

    # Calculate samples needed per category to roughly match good weather total
    total_categories = len(desired_categories)
    samples_per_category = good_weather_count // total_categories

    # Sample evenly from each category
    sampled_dfs = []
    for category in desired_categories:
        category_data = filtered_bad[filtered_bad['CATEGORY'] == category]
        if len(category_data) > samples_per_category:
            sampled = category_data.sample(n=samples_per_category, random_state=42)
        else:
            sampled = category_data  # Take all if we don't have enough
        sampled_dfs.append(sampled)

    # Combine sampled categories
    balanced_bad = pd.concat(sampled_dfs, ignore_index=True)

    print("\nBad weather category distribution after sampling:")
    print(balanced_bad['CATEGORY'].value_counts().to_markdown())

    return balanced_bad

# Filter and balance bad weather samples
balanced_bad_samples = filter_bad_weather_samples(bad_synthetic_samples)

#### Train Test Split

In [90]:
# Create labels (1 for bad weather, 0 for good weather)
good_synthetic_samples['NOGO_FLAG'] = 0
balanced_bad_samples['NOGO_FLAG'] = 1

In [91]:
# Combine synthetic datasets
synthetic_data = pd.concat([good_synthetic_samples, balanced_bad_samples], ignore_index=True)

In [92]:
# Combine synthetic and real data
X_combined = pd.concat([synthetic_data[features], launch_corpus[features]], axis=0)
y_combined = pd.concat([synthetic_data['NOGO_FLAG'], launch_corpus['NOGO_FLAG']], axis=0)

# Then do a regular train-test split
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, 
    y_combined,
    test_size=0.2,
    random_state=42,
    stratify=y_combined  # Maintain class distribution
)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
# Print basic information about X_train
print("X_train shape:", X_train.shape)
print("\nX_train dtypes:")
print(X_train.dtypes)

# Print a sample of the first row in a more readable format
print("\nFirst row sample:")
for feature in X_train.columns:
    print(f"\n{feature}:")
    print(f"Type: {type(X_train[feature].iloc[0])}")
    print(f"Shape: {X_train[feature].iloc[0].shape}")
    print(f"Values: {X_train[feature].iloc[0]}")

### Preprocessing

In [96]:
# Imports
from sklearn.preprocessing import StandardScaler

In [ ]:
def prepare_lstm_data(X_train, X_test, features):
    """
    Prepare data for LSTM model by:
    1. Standardizing features
    2. Reshaping arrays into 3D format [samples, time_steps, features]

    Args:
        X_train: Training data with array features
        X_test: Test data with array features
        features: List of feature names

    Returns:
        Scaled and reshaped training and test data
    """
    print("Starting data preparation for LSTM...")

    # Initialize lists to store processed data
    X_train_processed = []
    X_test_processed = []

    # Process each feature separately
    for feature in features:
        print(f"\nProcessing feature: {feature}")

        # Extract arrays from series and stack them
        train_arrays = np.stack(X_train[feature].values)
        test_arrays = np.stack(X_test[feature].values)

        # Reshape to 2D for scaling
        train_2d = train_arrays.reshape(-1, 1)
        test_2d = test_arrays.reshape(-1, 1)

        # Fit scaler on training data
        scaler = StandardScaler()
        scaler.fit(train_2d)

        # Transform both sets
        train_scaled = scaler.transform(train_2d)
        test_scaled = scaler.transform(test_2d)

        # Reshape back to original shape
        train_reshaped = train_scaled.reshape(train_arrays.shape)
        test_reshaped = test_scaled.reshape(test_arrays.shape)

        # Append to processed lists
        X_train_processed.append(train_reshaped)
        X_test_processed.append(test_reshaped)

        # Print feature statistics
        print(f"Training set - Mean: {train_scaled.mean():.3f}, Std: {train_scaled.std():.3f}")
        print(f"Test set - Mean: {test_scaled.mean():.3f}, Std: {test_scaled.std():.3f}")

    # Stack features to create final 3D arrays
    X_train_3d = np.stack(X_train_processed, axis=2)
    X_test_3d = np.stack(X_test_processed, axis=2)

    print("\nFinal array shapes:")
    print(f"Training set: {X_train_3d.shape} [samples, time_steps, features]")
    print(f"Test set: {X_test_3d.shape} [samples, time_steps, features]")

    return X_train_3d, X_test_3d

# Prepare the data
X_train_scaled, X_test_scaled = prepare_lstm_data(X_train, X_test, features)

# Verify the shapes and data
print("\nData Statistics:")
print(f"Training set - Mean: {X_train_scaled.mean():.3f}, Std: {X_train_scaled.std():.3f}")
print(f"Test set - Mean: {X_test_scaled.mean():.3f}, Std: {X_test_scaled.std():.3f}")

# Check for any remaining NaN or infinite values
print("\nQuality Check:")
print(f"Training set NaN count: {np.isnan(X_train_scaled).sum()}")
print(f"Training set Inf count: {np.isinf(X_train_scaled).sum()}")
print(f"Test set NaN count: {np.isnan(X_test_scaled).sum()}")
print(f"Test set Inf count: {np.isinf(X_test_scaled).sum()}")

In [ ]:
# Display sample of the prepared LSTM data
print("\nLSTM Data Sample:")
print("\nTraining Data (First 3 time steps, first 5 features of first sample):")
print(X_train_scaled[0, :3, :5])

print("\nTest Data (First 3 time steps, first 5 features of first sample):")
print(X_test_scaled[0, :3, :5])

# Show feature dimensions
print("\nFeature Names:")
for i, feature in enumerate(features):
    print(f"{i}: {feature}")


In [ ]:
print(launch_corpus.sample(20).to_markdown())

### Modeling

In [99]:
# Imports
import tensorflow as tf
import random
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

#### Model 1

Model

In [ ]:
# Version 1: Single Model

def create_lstm_model(input_shape, verbose=True):
    """
    Create LSTM model for weather prediction
    Args:
        input_shape: Tuple of (time_steps, n_features)
        verbose: Whether to print model summary
    """
    model = Sequential([
        # First LSTM layer
        # Return sequences=True because we're stacking LSTM layers
        LSTM(units=64, 
             return_sequences=True,
             input_shape=input_shape),
        Dropout(0.2),  # Prevent overfitting

        # Second LSTM layer
        LSTM(units=32, 
             return_sequences=False),  # False because it's the last LSTM layer
        Dropout(0.2),

        # Dense layers for final predictions
        Dense(16, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')  # Binary classification (go/no-go)
    ])

    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    if verbose:
        model.summary()

    return model

# Create early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Get input shape from our preprocessed data
input_shape = (X_train_scaled.shape[1], X_train_scaled.shape[2])  # (time_steps, features)

# Create and train model
model = create_lstm_model(input_shape)

# Train the model
history = model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

#### Model 2

Model

In [ ]:
# Version 2: K-Fold Cross Validation

def create_lstm_model(input_shape):
    """Recreate our LSTM model architecture"""
    model = Sequential([
        LSTM(units=64, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(units=32, return_sequences=False),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# First, ensure data is in the right format for LSTM
# Assuming each feature is already a sequence, we need to stack them
def prepare_data_for_lstm(X):
    # Stack the arrays
    arrays = []
    for feature in features:
        arrays.append(np.stack(X[feature].values))
    return np.stack(arrays, axis=2)

# Prepare all data
X_combined_3d = prepare_data_for_lstm(X_combined)

# Set up K-fold cross validation
n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Store results
cv_scores = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': []
}

# Get input shape from prepared data
input_shape = (X_combined_3d.shape[1], X_combined_3d.shape[2])  # (time_steps, features)

print(f"Starting {n_splits}-fold cross validation...")
print(f"Input shape: {input_shape}")

# Perform k-fold cross validation
for fold, (train_idx, val_idx) in enumerate(kfold.split(X_combined_3d)):
    print(f"\nFold {fold+1}/{n_splits}")

    # Split data
    X_train_fold = X_combined_3d[train_idx]
    X_val_fold = X_combined_3d[val_idx]
    y_train_fold = y_combined.iloc[train_idx]
    y_val_fold = y_combined.iloc[val_idx]

    # Create and train model
    model = create_lstm_model(input_shape)

    # Early stopping callback
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    # Train model
    history = model.fit(
        X_train_fold,
        y_train_fold,
        epochs=100,
        batch_size=32,
        validation_data=(X_val_fold, y_val_fold),
        callbacks=[early_stopping],
        verbose=0
    )

    # Evaluate model
    y_pred = model.predict(X_val_fold, verbose=0)
    y_pred_classes = (y_pred > 0.5).astype(int)

    # Calculate metrics
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

    cv_scores['accuracy'].append(accuracy_score(y_val_fold, y_pred_classes))
    cv_scores['precision'].append(precision_score(y_val_fold, y_pred_classes))
    cv_scores['recall'].append(recall_score(y_val_fold, y_pred_classes))
    cv_scores['f1'].append(f1_score(y_val_fold, y_pred_classes))

    print(f"Fold {fold+1} Accuracy: {cv_scores['accuracy'][-1]:.4f}")

# Print final results
print("\nCross-validation results:")
for metric in cv_scores:
    mean_score = np.mean(cv_scores[metric])
    std_score = np.std(cv_scores[metric])
    print(f"{metric.capitalize()}:")
    print(f"  Mean: {mean_score:.4f}")
    print(f"  Std:  {std_score:.4f}")

#### Model 3

In [102]:
from sklearn.metrics import confusion_matrix, classification_report

Imports

Utility Functions

In [103]:
def set_seeds(seed=42):
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

Model

In [ ]:
def create_lstm_model(input_shape, seed=42):
    """Create more stable LSTM model"""
    tf.random.set_seed(seed)

    model = Sequential([
        LSTM(units=64, 
             return_sequences=True,
             input_shape=input_shape,
             kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dropout(0.3),

        LSTM(units=32,
             return_sequences=False,
             kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dropout(0.3),

        Dense(16, 
              activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dropout(0.3),

        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# Prepare the data first
def prepare_data_for_lstm(X):
    arrays = []
    for feature in features:
        arrays.append(np.stack(X[feature].values))
    return np.stack(arrays, axis=2)

# Prepare train and test data
X_train_3d = prepare_data_for_lstm(X_train)
X_test_3d = prepare_data_for_lstm(X_test)

# Convert labels to numpy arrays
y_train_np = y_train.values
y_test_np = y_test.values

# Training with more patience
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

# Train multiple times to check stability
n_runs = 3
results = []

print("Input shape:", X_train_3d.shape)

for i in range(n_runs):
    print(f"\nRun {i+1}/{n_runs}")
    set_seeds(42 + i)  # Different seed each time

    model = create_lstm_model(input_shape=(X_train_3d.shape[1], X_train_3d.shape[2]))

    history = model.fit(
        X_train_3d, 
        y_train_np,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=0
    )

    loss, acc = model.evaluate(X_test_3d, y_test_np, verbose=0)
    results.append(acc)
    print(f"Run {i+1}: Accuracy = {acc:.4f}")

    # Get predictions and print classification report
    y_pred = model.predict(X_test_3d, verbose=0)
    y_pred_classes = (y_pred > 0.5).astype(int)
    print("\nClassification Report:")
    print(classification_report(y_test_np, y_pred_classes))

print(f"\nMean Accuracy: {np.mean(results):.4f} ± {np.std(results):.4f}")

### Evaluation

In [105]:
# Imports
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Performance Metrics

# 1. Plot training history
def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
 
    # Loss plot
    ax1.plot(history.history['loss'], label='Training Loss')
    ax1.plot(history.history['val_loss'], label='Validation Loss')
    ax1.set_title('Model Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()

    # Accuracy plot
    ax2.plot(history.history['accuracy'], label='Training Accuracy')
    ax2.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax2.set_title('Model Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()

    plt.tight_layout()
    plt.show()

# 2. Evaluate on test data
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# 3. Generate predictions and confusion matrix
y_pred = model.predict(X_test_scaled)
y_pred_classes = (y_pred > 0.5).astype(int)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))

# Plot confusion matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Plot training history
plot_training_history(history)

In [ ]:
# Evaluating CV Results

# 1. Plot CV scores
# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 1. Box Plot
scores_df = pd.DataFrame(cv_scores)
sns.boxplot(data=scores_df, ax=ax1)
ax1.set_title('Distribution of Metrics Across Folds (Box Plot)')
ax1.set_ylabel('Score')
ax1.grid(True, alpha=0.3)

# 2. Violin Plot (shows full distribution)
sns.violinplot(data=scores_df, ax=ax2)
ax2.set_title('Distribution of Metrics Across Folds (Violin Plot)')
ax2.set_ylabel('Score')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Also show individual fold scores in a table
print("\nDetailed scores per fold:")
fold_df = pd.DataFrame({
    'Fold': range(1, n_splits + 1),
    'Accuracy': cv_scores['accuracy'],
    'Precision': cv_scores['precision'],
    'Recall': cv_scores['recall'],
    'F1': cv_scores['f1']
}).set_index('Fold')

print(fold_df.round(4).to_markdown())

# Calculate and display statistical summary
print("\nStatistical Summary:")
print(fold_df.describe().round(4).to_markdown())

### Post-Hoc Analysis

Feature Importance

In [110]:
# Imports
from sklearn.inspection import permutation_importance

In [ ]:
def analyze_feature_importance(model, X_test_3d, y_test, features):
    """Analyze feature importance using mean absolute impact on predictions"""

    # Initialize importance scores
    importance_scores = []
    importance_stds = []

    # Get baseline predictions
    baseline_pred = model.predict(X_test_3d)

    # For each feature
    for feat_idx in range(X_test_3d.shape[2]):  # Loop through features
        # Create copies for permutation
        importance_per_run = []

        # Repeat multiple times for stability
        for _ in range(10):  # number of repeats
            # Create a copy of the test data
            X_permuted = X_test_3d.copy()

            # Shuffle the feature across all time steps
            for time_step in range(X_test_3d.shape[1]):
                X_permuted[:, time_step, feat_idx] = np.random.permutation(
                    X_permuted[:, time_step, feat_idx]
                )

            # Get predictions with permuted feature
            permuted_pred = model.predict(X_permuted)

            # Calculate impact (mean absolute difference in predictions)
            impact = np.mean(np.abs(baseline_pred - permuted_pred))
            importance_per_run.append(impact)

        # Store mean and std of importance
        importance_scores.append(np.mean(importance_per_run))
        importance_stds.append(np.std(importance_per_run))

    # Create importance DataFrame
    importance_df = pd.DataFrame({
        'Feature': features,
        'Importance_Mean': importance_scores,
        'Importance_Std': importance_stds
    }).sort_values('Importance_Mean', ascending=False)

    # Plot feature importances
    plt.figure(figsize=(12, 8))
    plt.errorbar(
        importance_df['Importance_Mean'],
        range(len(features)),
        xerr=importance_df['Importance_Std'],
        fmt='o'
    )
    plt.yticks(range(len(features)), importance_df['Feature'])
    plt.xlabel('Feature Importance (Mean Absolute Impact)')
    plt.title('Feature Importance Analysis')
    plt.tight_layout()
    plt.show()

    return importance_df

# Analyze feature importance
analyze_feature_importance(model, X_test_3d, y_test, features)

Surrogate Decision Tree

Analyze Predictions with Original Set

In [ ]:
def predict_on_launch_corpus(model, launch_corpus, features):
    """
    Make predictions on launch corpus data
    """
    # Prepare data in 3D format for LSTM
    arrays = []
    for feature in features:
        feature_array = np.stack(launch_corpus[feature].values)
        arrays.append(feature_array)
    X = np.stack(arrays, axis=2)

    # Get predictions
    y_pred_proba = model.predict(X)
    y_pred = (y_pred_proba > 0.5).astype(int)

    # Create results DataFrame
    results_df = launch_corpus.copy()
    results_df['Predicted_NOGO'] = y_pred.flatten()  # Flatten predictions
    results_df['Prediction_Confidence'] = y_pred_proba.flatten()  # Flatten probabilities

    # Add correct/incorrect flag
    results_df['Correct_Prediction'] = results_df['Predicted_NOGO'] == results_df['NOGO_FLAG']

    # Print analysis
    print("\nPrediction Analysis:")
    print(f"Total cases: {len(results_df)}")
    print(f"Correct predictions: {results_df['Correct_Prediction'].sum()}")
    print(f"Accuracy: {results_df['Correct_Prediction'].mean():.2%}")

    # Confusion Matrix
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(results_df['NOGO_FLAG'], results_df['Predicted_NOGO'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    # Analyze interesting cases
    incorrect = results_df[~results_df['Correct_Prediction']]
    print("\nIncorrect Predictions Analysis:")
    print(f"Total incorrect: {len(incorrect)}")

    if len(incorrect) > 0:
        print("\nSample of incorrect predictions:")
        for idx, row in incorrect.head().iterrows():
            print(f"\nCase {idx}:")
            print(f"Date: {row['DATE']}")
            print(f"Launch Vehicle: {row['LAUNCH_VEHICLE']}")
            print(f"Predicted: {'NOGO' if row['Predicted_NOGO'] else 'GO'}")
            print(f"Actual: {'NOGO' if row['NOGO_FLAG'] else 'GO'}")
            print(f"Confidence: {row['Prediction_Confidence']:.3f}")  # No need for [0]
            print(f"Remarks: {row['REMARKS']}")

    # Additional Analysis
    print("\nDetailed Statistics:")
    print("Average confidence in correct predictions: "
          f"{results_df[results_df['Correct_Prediction']]['Prediction_Confidence'].mean():.3f}")
    print("Average confidence in incorrect predictions: "
          f"{results_df[~results_df['Correct_Prediction']]['Prediction_Confidence'].mean():.3f}")

    # Plot confidence distribution
    plt.figure(figsize=(10, 6))
    plt.hist(results_df['Prediction_Confidence'], bins=20, alpha=0.5, label='All predictions')
    plt.hist(results_df[~results_df['Correct_Prediction']]['Prediction_Confidence'], 
             bins=20, alpha=0.5, label='Incorrect predictions')
    plt.title('Distribution of Prediction Confidence')
    plt.xlabel('Confidence')
    plt.ylabel('Count')
    plt.legend()
    plt.show()

    return results_df

# Run predictions on launch corpus
results = predict_on_launch_corpus(model, launch_corpus, features)

# Optional: Save results to CSV
# results.to_csv('launch_predictions.csv', index=True)

In [ ]:
print(results.head().to_markdown())

In [ ]:
def get_reason_codes(model, X, features, threshold=0.1):
    """
    Generate reason codes by analyzing feature importance for each prediction
    """
    # Get baseline prediction
    baseline_pred = model.predict(X)
    
    # Store feature impacts
    feature_impacts = np.zeros((X.shape[0], len(features)))
    
    # For each feature, measure its impact
    for i, feature in enumerate(features):
        # Create copy with feature zeroed out
        X_modified = X.copy()
        X_modified[:, :, i] = 0
        
        # Get new prediction
        new_pred = model.predict(X_modified)
        
        # Impact is difference from baseline
        feature_impacts[:, i] = np.abs(baseline_pred - new_pred).flatten()
    
    return feature_impacts

def predict_with_reasons(model, launch_corpus, features, n_reasons=3):
    """
    Make predictions with reason codes
    """
    # Prepare data
    arrays = []
    for feature in features:
        feature_array = np.stack(launch_corpus[feature].values)
        arrays.append(feature_array)
    X = np.stack(arrays, axis=2)
    
    # Get predictions
    y_pred_proba = model.predict(X)
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Get feature impacts
    feature_impacts = get_reason_codes(model, X, features)
    
    # Create results DataFrame
    results_df = launch_corpus.copy()
    results_df['Predicted_NOGO'] = y_pred.flatten()
    results_df['Prediction_Confidence'] = y_pred_proba.flatten()
    results_df['Correct_Prediction'] = results_df['Predicted_NOGO'] == results_df['NOGO_FLAG']
    
    # Add reason codes
    for i in range(len(results_df)):  # Use numeric index instead of DataFrame index
        impacts = feature_impacts[i]
        # Get top N reasons
        top_reasons = np.argsort(impacts)[-n_reasons:][::-1]
        reasons = []
        for reason_idx in top_reasons:
            feature = features[reason_idx]
            impact = impacts[reason_idx]
            if impact > 0.1:  # Only include significant impacts
                reasons.append(f"{feature} ({impact:.3f})")
        results_df.iloc[i, results_df.columns.get_loc('Reason_Codes')] = '; '.join(reasons)
    
    # Print analysis
    print("\nPrediction Analysis:")
    print(f"Total cases: {len(results_df)}")
    print(f"Correct predictions: {results_df['Correct_Prediction'].sum()}")
    print(f"Accuracy: {results_df['Correct_Prediction'].mean():.2%}")
    
    # Confusion Matrix
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(results_df['NOGO_FLAG'], results_df['Predicted_NOGO'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
    # Sample predictions with reasons
    print("\nSample Predictions with Reasons:")
    nogo_preds = results_df[results_df['Predicted_NOGO'] == 1]
    if len(nogo_preds) > 0:
        for _, row in nogo_preds.head().iterrows():
            print(f"\nDate: {row['DATE']}")
            print(f"Predicted: {'NOGO' if row['Predicted_NOGO'] else 'GO'}")
            print(f"Confidence: {row['Prediction_Confidence']:.3f}")
            print(f"Actual: {'NOGO' if row['NOGO_FLAG'] else 'GO'}")
            print("Reason Codes:")
            if isinstance(row['Reason_Codes'], str):
                for reason in row['Reason_Codes'].split(';'):
                    print(f"  - {reason}")
            print(f"Remarks: {row['REMARKS']}")
    
    return results_df

# First add the Reason_Codes column
launch_corpus['Reason_Codes'] = ''

# Run predictions with reason codes
results = predict_with_reasons(model, launch_corpus, features)

# Optional: Save detailed results
# results.to_csv('launch_predictions_with_reasons.csv', index=True)

In [ ]:
def predict_with_reasons(model, launch_corpus, features, n_reasons=3, confidence_threshold=0.5):
    """
    Make predictions with reasons and adjustable confidence threshold
    """
    # Prepare data
    arrays = []
    for feature in features:
        feature_array = np.stack(launch_corpus[feature].values)
        arrays.append(feature_array)
    X = np.stack(arrays, axis=2)
    
    # Get predictions
    y_pred_proba = model.predict(X)
    
    # Try different confidence thresholds
    thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]
    print("\nTesting different confidence thresholds:")
    
    for threshold in thresholds:
        y_pred = (y_pred_proba > threshold).astype(int)
        cm = confusion_matrix(launch_corpus['NOGO_FLAG'], y_pred)
        
        # Calculate metrics
        tn, fp, fn, tp = cm.ravel()
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        print(f"\nThreshold: {threshold}")
        print(f"Accuracy: {accuracy:.3f}")
        print(f"Precision: {precision:.3f}")
        print(f"Recall: {recall:.3f}")
        print(f"F1 Score: {f1:.3f}")
        print("Confusion Matrix:")
        print(cm)
    
    # Use best threshold based on F1 score
    best_threshold = 0.5  # Will be updated based on results
    y_pred = (y_pred_proba > best_threshold).astype(int)
    
    # Create results DataFrame
    results_df = launch_corpus.copy()
    results_df['Predicted_NOGO'] = y_pred.flatten()
    results_df['Prediction_Confidence'] = y_pred_proba.flatten()
    results_df['Correct_Prediction'] = results_df['Predicted_NOGO'] == results_df['NOGO_FLAG']
    
    # Print class distribution
    print("\nClass Distribution:")
    print("Actual NOGO:", sum(launch_corpus['NOGO_FLAG']))
    print("Predicted NOGO:", sum(y_pred))
    
    # Print high confidence predictions
    high_conf = results_df[results_df['Prediction_Confidence'] > 0.8]
    print(f"\nHigh confidence predictions (>0.8): {len(high_conf)}")
    print("High confidence accuracy:", 
          (high_conf['Correct_Prediction'].sum() / len(high_conf) if len(high_conf) > 0 else 0))
    
    return results_df

# Run analysis with different thresholds
results = predict_with_reasons(model, launch_corpus, features)

# Optional: Retrain model with class weights
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(launch_corpus['NOGO_FLAG']),
    y=launch_corpus['NOGO_FLAG']
)

print("\nSuggested class weights:")
for i, weight in enumerate(class_weights):
    print(f"Class {i}: {weight:.2f}")